In [ ]:
import random
from llama_cpp import Llama
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# Function to load and process the text from the user manual
def load_and_process_manual(file_path):
    # Load the text file
    loader = TextLoader(file_path)
    documents = loader.load()
    
    # Split the text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    chunks = text_splitter.split_documents(documents)
    
    print(f"Bedienungsanleitung geladen und in {len(chunks)} Teile aufgeteilt")
    return chunks

# Function to create vector store from document chunks
def create_vector_store(chunks):
    # Initialize the embeddings model - using a German model
    embeddings = HuggingFaceEmbeddings(
        model_name="deutsche-telekom/gbert-large",
        model_kwargs={"device": "cpu"},
    )
    
    # Create the vector store
    vector_store = FAISS.from_documents(chunks, embeddings)
    
    print("Vektordatenbank erfolgreich erstellt")
    return vector_store

# Function to retrieve relevant context based on step instruction
def retrieve_context(vector_store, step_instruction, k=3):
    # Retrieve the most similar chunks to the step instruction
    relevant_chunks = vector_store.similarity_search(step_instruction, k=k)
    
    # Concatenate the content of the chunks
    context = "\n\n".join([doc.page_content for doc in relevant_chunks])
    
    return context

# Initialize the model with appropriate settings
llm = Llama(
    model_path="em_german_7b_v01.Q8_0.gguf",
    n_ctx=2048,
    verbose=False,
    n_batch=512,
    echo=False,
    temperature=0.4
)

# Define the workflow steps - in German since we're using a German model
workflow_steps = [
    "Montieren Sie die Halterung und das notwendige Zubehör.",
    "Positionieren und schließen Sie das CARDIOHELP-Gerät an.",
    "Verbinden Sie die Sensoren.",
    "Schalten Sie das CARDIOHELP-Gerät ein und lassen Sie es einen Selbsttest durchführen."
]

def interpret_user_response(user_input):
    """Simplified function that checks for confirmation words in German"""
    user_input = user_input.lower()
   
    # List of German confirmation words
    confirmation_words = [
        "fertig", "erledigt", "abgeschlossen", "ok", "ja",
        "gemacht", "geschafft", "bereit", "weiter"]
   
    # Check if any confirmation word is in the user input
    for word in confirmation_words:
        if word in user_input:
            return "yes"
   
    return "no"

# Main workflow
def main():
    print("CARDIOHELP-Einrichtungsassistent\n")
    print("Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.")
    print("--------------------------------------------------\n")
    
    # Load and prepare the manual
    manual_path = "bedienungsanleitung.txt"
    if not os.path.exists(manual_path):
        print(f"Warnung: Bedienungsanleitung '{manual_path}' nicht gefunden. Fortfahren ohne Kontext.")
        vector_store = None
    else:
        chunks = load_and_process_manual(manual_path)
        vector_store = create_vector_store(chunks)
    
    current_step = 0
    
    while current_step < len(workflow_steps):
        step_instruction = workflow_steps[current_step]
        
        # System prompt
        system_prompt = f"""
        Du bist ein hilfreicher Assistent für die Einrichtung des CARDIOHELP-Medizingeräts.
       
        Aktueller Status: Schritt {current_step + 1} von {len(workflow_steps)}
       
        Sei präzise und klar in deinen Anweisungen.
        """
        
        # Add relevant context from the manual if available
        if vector_store:
            context = retrieve_context(vector_store, step_instruction)
            system_prompt += f"\n\nRelevante Informationen aus der Bedienungsanleitung:\n{context}"
        
        full_prompt = f"{system_prompt}\n\nBitte gib detaillierte Anweisungen für Schritt {current_step + 1}: {step_instruction}"
       
        try:
            # Generate a response
            response = llm(
                full_prompt,
                max_tokens=512,
                temperature=0.4,
                stop=["</s>", "<s>"]
            )
            response_text = response['choices'][0]['text'].strip()
           
            # Display the instructions
            print(f"\n--- Schritt {current_step + 1}: {step_instruction} ---")
            print(response_text)
            
            # Get user confirmation to proceed (in a real application)
            # Here we can use input() for a real user, or random choice for testing
            print("\nBitte bestätigen Sie, wenn Sie fertig sind (sagen Sie 'weiter', 'fertig', 'ok', etc.)")
            
            # For testing - in a real app, use: user_input = input("> ")
            user_input = random.choice(["weiter", "fertig", "ok", "ja", "erledigt"])
            print(f"> {user_input}")
            
            # Check if user confirmed completion
            decision = interpret_user_response(user_input)
            if decision == "yes":
                current_step += 1
            else:
                print("Ich verstehe nicht. Bitte bestätigen Sie, wenn Sie fertig sind.")
               
        except Exception as e:
            print(f"Fehler: {e}")
            # Increment step to avoid infinite loop in case of error
            current_step += 1
    
    print("\nEinrichtung abgeschlossen!")

if __name__ == "__main__":
    main()

In [3]:
import random
from llama_cpp import Llama
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings

# The compact manual content provided
MANUAL_CONTENT = """CARDIOHELP System-Vorbereitung in 4 Schritten
Schritt 1: Montage der Komponenten
Halterung für CARDIOHELP-i montieren: HKH 9102-M für 38mm Mastdurchmesser verwenden
Transportzubehör vorbereiten: Transporthalter HKH 8860, CARDIOHELP Emergency Drive in Transportbox sichern
CARDIOHELP Emergency Drive positionieren: Halterung für Notantrieb montieren (HKH 9101-R/M)
CARDIOHELP-i aufstellen: Gerät auf Halterung befestigen, Schutzbügel öffnen, Stromversorgung anschließen
Schritt 2: Anschluss von Sensoren und externen Systemen
Sensoren anschließen: Stecker anhand der roten Markierungen ausrichten
Externe Geräte verbinden: Datenaufzeichnungssystem über USB-Typ-B, externes Alarmsystem
Venösen Messkopf anschließen: An "Venöser Messkopf"-Anschluss und in Halterung platzieren
Sensorkabel für Einmalprodukt verbinden: Vor dem Einschalten anschließen für korrekte Erkennung
Schritt 3: Einschalten und Systemkonfiguration
Einschalten und Selbsttest: Venösen Messkopf vor Einschalten sichern, "Ein/Aus"-Taste drücken
Grundeinstellungen konfigurieren: Helligkeit, Lautstärke, Sprache, Zeit/Datum einstellen
Systeminformationen prüfen: Software-Version, Betriebsstunden, Batteriestatus kontrollieren
Key-User-Funktionen (passwortgeschützt): Alarmkonfiguration, Klinikkonfiguration speichern
Schritt 4: Anwendungsvorbereitung und Funktionstest
Perfusion vorbereiten: Einmalprodukt montieren, integrierte Sensorik anschließen
Erforderliche Sensoren positionieren: Fluss-/Blasensensor, Temperatursensoren, Drucksensoren
Funktionstests durchführen: Blasenüberwachung, Niveauüberwachung, Messkopf-Status prüfen
Kalibrierungen vornehmen: Nullkalibrierung für Drucksensoren, Fluss-Off-Set-Kalibrierung
Abschließende Überprüfung: Sicherheitssiegel, Bedienelemente, Sensorfunktionen, Notantrieb"""

# Function to process the manual text directly from content string
def process_manual_content():
    # Create a single Document object
    doc = Document(page_content=MANUAL_CONTENT, metadata={"source": "CARDIOHELP-Manual"})
    
    # Split the text into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,  # Smaller chunk size due to compact manual
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", ":", " ", ""]
    )
    chunks = text_splitter.split_documents([doc])
    
    print(f"Bedienungsanleitung in {len(chunks)} Teile aufgeteilt")
    return chunks


def create_vector_store(chunks):
    try:
        # Try using a different, more accessible HuggingFace model
        # This model is well-supported and commonly available
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
            model_kwargs={"device": "cpu"},
        )
        
        # Create the vector store
        vector_store = FAISS.from_documents(chunks, embeddings)
        
        print("Vektordatenbank erfolgreich erstellt mit multilingual model")
        return vector_store
        
    except Exception as e:
        print(f"Fehler mit dem ersten Modell: {e}")
        print("Versuche alternatives Modell...")
        
        try:
            # Try another fallback model
            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2",
                model_kwargs={"device": "cpu"},
            )
            
            # Create the vector store
            vector_store = FAISS.from_documents(chunks, embeddings)
            
            print("Vektordatenbank erfolgreich erstellt mit alternativem Modell")
            return vector_store
            
        except Exception as e:
            print(f"Fehler mit dem zweiten Modell: {e}")
            print("Verwende lokale Embedding-Methode als Fallback")
            
            # If both models fail, use a basic TF-IDF vectorizer as fallback
            from sklearn.feature_extraction.text import TfidfVectorizer
            from sklearn.metrics.pairwise import cosine_similarity
            
            # Extract text from documents
            texts = [doc.page_content for doc in chunks]
            
            # Create TF-IDF vectors
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(texts)
            
            # Wrap in a simple class to mimic the FAISS interface
            class TfidfStore:
                def __init__(self, matrix, vectorizer, docs):
                    self.matrix = matrix
                    self.vectorizer = vectorizer
                    self.docs = docs
                
                def similarity_search(self, query, k=2):
                    # Transform query
                    query_vec = self.vectorizer.transform([query])
                    
                    # Calculate similarity
                    similarity = cosine_similarity(query_vec, self.matrix).flatten()
                    
                    # Get top k indices
                    indices = similarity.argsort()[:-k-1:-1]
                    
                    # Return top k documents
                    return [self.docs[i] for i in indices]
            
            tfidf_store = TfidfStore(tfidf_matrix, vectorizer, chunks)
            print("Vektordatenbank erfolgreich erstellt mit TF-IDF fallback")
            return tfidf_store

# Function to retrieve relevant context based on step instruction
def retrieve_context(vector_store, step_instruction, k=2):
    # Retrieve the most similar chunks to the step instruction
    relevant_chunks = vector_store.similarity_search(step_instruction, k=k)
    
    # Concatenate the content of the chunks
    context = "\n\n".join([doc.page_content for doc in relevant_chunks])
    
    return context

# Initialize the model with appropriate settings
llm = Llama(
    model_path="em_german_7b_v01.Q8_0.gguf",
    n_ctx=2048,  # Reduced context size due to smaller manual
    verbose=False,
    n_batch=256,
    echo=False,
    temperature=0.3
)

# Define the workflow steps directly from the manual sections
workflow_steps = [
    "Montage der Komponenten",
    "Anschluss von Sensoren und externen Systemen",
    "Einschalten und Systemkonfiguration",
    "Anwendungsvorbereitung und Funktionstest"
]

def interpret_user_response(user_input):
    """Function that checks for confirmation words in German"""
    user_input = user_input.lower()
   
    # List of German confirmation words
    confirmation_words = [
        "fertig", "erledigt", "abgeschlossen", "ok", "ja",
        "gemacht", "geschafft", "bereit", "weiter"]
   
    # Check if any confirmation word is in the user input
    for word in confirmation_words:
        if word in user_input:
            return "yes"
   
    return "no"

# Main workflow
def main():
    print("CARDIOHELP-Einrichtungsassistent\n")
    print("Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.")
    print("--------------------------------------------------\n")
    
    # Process manual content and create vector store
    chunks = process_manual_content()
    vector_store = create_vector_store(chunks)
    
    current_step = 0
    
    while current_step < len(workflow_steps):
        step_instruction = workflow_steps[current_step]
        
        # Get relevant content from the manual for this step
        context = retrieve_context(vector_store, step_instruction)
        
        # System prompt
        system_prompt = f"""
        Du bist ein hilfreicher Assistent für die Einrichtung des CARDIOHELP-Medizingeräts.
       
        Aktueller Status: Schritt {current_step + 1} von {len(workflow_steps)}
        Schritt: {step_instruction}
       
        Sei präzise und klar in deinen Anweisungen. Gib detaillierte Anleitung zur Durchführung dieses Schritts in Relevanten Informationen aus der Bedienungsanleitung.
        
        Relevante Informationen aus der Bedienungsanleitung:
        {context}
        """
        
        try:
            # Generate a response
            response = llm(
                system_prompt,
                max_tokens=256,  # Reduced for shorter responses
                temperature=0.3,  # Lower temperature for more precise responses
                stop=["</s>", "<s>"],
            )
            response_text = response['choices'][0]['text'].strip()
           
            # Display the instructions
            print(f"\n--- Schritt {current_step + 1}: {step_instruction} ---")
            print(response_text)
            
            # Get user confirmation to proceed
            print("\nBitte bestätigen Sie, wenn Sie fertig sind (sagen Sie 'weiter', 'fertig', 'ok', etc.)")
            
            # For testing - in a real app, use: user_input = input("> ")
            user_input = random.choice(["weiter", "fertig", "ok", "ja", "erledigt"])
            print(f"> {user_input}")
            
            # Check if user confirmed completion
            decision = interpret_user_response(user_input)
            if decision == "yes":
                current_step += 1
                if current_step < len(workflow_steps):
                    print(f"\nWir gehen nun zu Schritt {current_step + 1} über.")
            else:
                print("Ich verstehe nicht. Bitte bestätigen Sie, wenn Sie fertig sind.")
               
        except Exception as e:
            print(f"Fehler bei der Verarbeitung: {e}")
            # Increment step to avoid infinite loop in case of error
            current_step += 1
    
    print("\nCARDIOHELP-Einrichtung abgeschlossen! Das Gerät ist nun einsatzbereit.")

if __name__ == "__main__":
    main()

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


CARDIOHELP-Einrichtungsassistent

Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.
--------------------------------------------------

Bedienungsanleitung in 11 Teile aufgeteilt
Vektordatenbank erfolgreich erstellt mit multilingual model

--- Schritt 1: Montage der Komponenten ---
Bitte gib deine Anweisungen für jeden Schritt an.

Bitte bestätigen Sie, wenn Sie fertig sind (sagen Sie 'weiter', 'fertig', 'ok', etc.)
> erledigt

Wir gehen nun zu Schritt 2 über.

--- Schritt 2: Anschluss von Sensoren und externen Systemen ---
Bitte gib detaillierte Anleitung zur Durchführung dieses Schritts in Relevanten Informationen aus der Bedienungsanleitung an.

Bitte bestätigen Sie, wenn Sie fertig sind (sagen Sie 'weiter', 'fertig', 'ok', etc.)
> ja

Wir gehen nun zu Schritt 3 über.

--- Schritt 3: Einschalten und Systemkonfiguration ---
Achte auf:
        - Batteriestatus des Messkopfes
        - Software-Version
        - Betriebsstunden


In [8]:
import random
from llama_cpp import Llama
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# The compact manual content provided
MANUAL_CONTENT = """CARDIOHELP System-Vorbereitung in 4 Schritten
Schritt 1: Montage der Komponenten
Halterung für CARDIOHELP-i montieren: HKH 9102-M für 38mm Mastdurchmesser verwenden
Transportzubehör vorbereiten: Transporthalter HKH 8860, CARDIOHELP Emergency Drive in Transportbox sichern
CARDIOHELP Emergency Drive positionieren: Halterung für Notantrieb montieren (HKH 9101-R/M)
CARDIOHELP-i aufstellen: Gerät auf Halterung befestigen, Schutzbügel öffnen, Stromversorgung anschließen
Schritt 2: Anschluss von Sensoren und externen Systemen
Sensoren anschließen: Stecker anhand der roten Markierungen ausrichten
Externe Geräte verbinden: Datenaufzeichnungssystem über USB-Typ-B, externes Alarmsystem
Venösen Messkopf anschließen: An "Venöser Messkopf"-Anschluss und in Halterung platzieren
Sensorkabel für Einmalprodukt verbinden: Vor dem Einschalten anschließen für korrekte Erkennung
Schritt 3: Einschalten und Systemkonfiguration
Einschalten und Selbsttest: Venösen Messkopf vor Einschalten sichern, "Ein/Aus"-Taste drücken
Grundeinstellungen konfigurieren: Helligkeit, Lautstärke, Sprache, Zeit/Datum einstellen
Systeminformationen prüfen: Software-Version, Betriebsstunden, Batteriestatus kontrollieren
Key-User-Funktionen (passwortgeschützt): Alarmkonfiguration, Klinikkonfiguration speichern
Schritt 4: Anwendungsvorbereitung und Funktionstest
Perfusion vorbereiten: Einmalprodukt montieren, integrierte Sensorik anschließen
Erforderliche Sensoren positionieren: Fluss-/Blasensensor, Temperatursensoren, Drucksensoren
Funktionstests durchführen: Blasenüberwachung, Niveauüberwachung, Messkopf-Status prüfen
Kalibrierungen vornehmen: Nullkalibrierung für Drucksensoren, Fluss-Off-Set-Kalibrierung
Abschließende Überprüfung: Sicherheitssiegel, Bedienelemente, Sensorfunktionen, Notantrieb"""

# Function to process the manual text directly from content string
def process_manual_content():
    # Create a single Document object
    doc = Document(page_content=MANUAL_CONTENT, metadata={"source": "CARDIOHELP-Manual"})
    
    # Split the text into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,  # Smaller chunk size due to compact manual
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", ":", " ", ""]
    )
    chunks = text_splitter.split_documents([doc])
    
    print(f"Bedienungsanleitung in {len(chunks)} Teile aufgeteilt")
    return chunks


def create_vector_store(chunks):
    try:
        # Using multilingual model for better German support
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
            model_kwargs={"device": "cpu"},
        )
        
        # Create the vector store
        vector_store = FAISS.from_documents(chunks, embeddings)
        
        print("Vektordatenbank erfolgreich erstellt mit multilingual model")
        return vector_store
        
    except Exception as e:
        print(f"Fehler mit dem ersten Modell: {e}")
        print("Versuche alternatives Modell...")
        
        try:
            # Try another fallback model
            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2",
                model_kwargs={"device": "cpu"},
            )
            
            # Create the vector store
            vector_store = FAISS.from_documents(chunks, embeddings)
            
            print("Vektordatenbank erfolgreich erstellt mit alternativem Modell")
            return vector_store
            
        except Exception as e:
            print(f"Fehler mit dem zweiten Modell: {e}")
            print("Verwende lokale Embedding-Methode als Fallback")
            
            # If both models fail, use a basic TF-IDF vectorizer as fallback
            from sklearn.feature_extraction.text import TfidfVectorizer
            from sklearn.metrics.pairwise import cosine_similarity
            
            # Extract text from documents
            texts = [doc.page_content for doc in chunks]
            
            # Create TF-IDF vectors
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(texts)
            
            # Wrap in a simple class to mimic the FAISS interface
            class TfidfStore:
                def __init__(self, matrix, vectorizer, docs):
                    self.matrix = matrix
                    self.vectorizer = vectorizer
                    self.docs = docs
                
                def similarity_search(self, query, k=2):
                    # Transform query
                    query_vec = self.vectorizer.transform([query])
                    
                    # Calculate similarity
                    similarity = cosine_similarity(query_vec, self.matrix).flatten()
                    
                    # Get top k indices
                    indices = similarity.argsort()[:-k-1:-1]
                    
                    # Return top k documents
                    return [self.docs[i] for i in indices]
            
            tfidf_store = TfidfStore(tfidf_matrix, vectorizer, chunks)
            print("Vektordatenbank erfolgreich erstellt mit TF-IDF fallback")
            return tfidf_store

# Function to retrieve relevant context based on step instruction
def retrieve_context(vector_store, step_instruction, k=2):
    # Retrieve the most similar chunks to the step instruction
    relevant_chunks = vector_store.similarity_search(step_instruction, k=k)
    
    # Concatenate the content of the chunks
    context = "\n\n".join([doc.page_content for doc in relevant_chunks])
    
    return context

# Dictionary of pre-defined instructions for each step
# This helps overcome the LLM's tendency to repeat prompts
STEP_INSTRUCTIONS = {
    "Montage der Komponenten": """
Bitte führen Sie die folgenden Schritte zur Montage der Komponenten aus:

1. Halterung für CARDIOHELP-i montieren:
   - Verwenden Sie die Halterung HKH 9102-M für 38mm Mastdurchmesser
   
2. Transportzubehör vorbereiten:
   - Bereiten Sie den Transporthalter HKH 8860 vor
   - Sichern Sie den CARDIOHELP Emergency Drive in der Transportbox
   
3. CARDIOHELP Emergency Drive positionieren:
   - Montieren Sie die Halterung für den Notantrieb (HKH 9101-R/M)
   
4. CARDIOHELP-i aufstellen:
   - Befestigen Sie das Gerät auf der Halterung
   - Öffnen Sie den Schutzbügel
   - Schließen Sie die Stromversorgung an
    """,
    
    "Anschluss von Sensoren und externen Systemen": """
Bitte führen Sie die folgenden Schritte zum Anschluss von Sensoren und externen Systemen durch:

1. Sensoren anschließen:
   - Richten Sie alle Stecker anhand der roten Markierungen aus
   
2. Externe Geräte verbinden:
   - Verbinden Sie das Datenaufzeichnungssystem über den USB-Typ-B Anschluss
   - Schließen Sie das externe Alarmsystem an
   
3. Venösen Messkopf anschließen:
   - Verbinden Sie ihn mit dem "Venöser Messkopf"-Anschluss
   - Platzieren Sie ihn korrekt in der vorgesehenen Halterung
   
4. Sensorkabel für Einmalprodukt verbinden:
   - Wichtig: Diese müssen VOR dem Einschalten angeschlossen werden
   - Achten Sie auf die korrekte Ausrichtung für die Geräteerkennung
    """,
    
    "Einschalten und Systemkonfiguration": """
Bitte führen Sie die folgenden Schritte zum Einschalten und zur Systemkonfiguration durch:

1. Einschalten und Selbsttest:
   - Stellen Sie sicher, dass der venöse Messkopf vor dem Einschalten gesichert ist
   - Drücken Sie die "Ein/Aus"-Taste und warten Sie auf den Selbsttest
   
2. Grundeinstellungen konfigurieren:
   - Stellen Sie die gewünschte Helligkeit ein
   - Passen Sie die Lautstärke an
   - Wählen Sie die gewünschte Sprache
   - Konfigurieren Sie Zeit und Datum
   
3. Systeminformationen prüfen:
   - Kontrollieren Sie die Software-Version
   - Überprüfen Sie die Betriebsstunden
   - Überprüfen Sie den Batteriestatus
   
4. Key-User-Funktionen (passwortgeschützt):
   - Konfigurieren Sie die Alarmeinstellungen
   - Speichern Sie die Klinikkonfiguration
    """,
    
    "Anwendungsvorbereitung und Funktionstest": """
Bitte führen Sie die folgenden Schritte zur Anwendungsvorbereitung und zum Funktionstest durch:

1. Perfusion vorbereiten:
   - Montieren Sie das Einmalprodukt gemäß Anleitung
   - Schließen Sie die integrierte Sensorik an
   
2. Erforderliche Sensoren positionieren:
   - Platzieren Sie den Fluss-/Blasensensor korrekt
   - Positionieren Sie alle Temperatursensoren
   - Bringen Sie die Drucksensoren an den vorgesehenen Stellen an
   
3. Funktionstests durchführen:
   - Testen Sie die Blasenüberwachung
   - Überprüfen Sie die Niveauüberwachung
   - Kontrollieren Sie den Status des Messkopfs
   
4. Kalibrierungen vornehmen:
   - Führen Sie die Nullkalibrierung für Drucksensoren durch
   - Kalibrieren Sie den Fluss-Off-Set
   
5. Abschließende Überprüfung:
   - Kontrollieren Sie alle Sicherheitssiegel
   - Testen Sie sämtliche Bedienelemente
   - Überprüfen Sie alle Sensorfunktionen
   - Testen Sie den Notantrieb
    """
}

# Initialize the model with appropriate settings
def initialize_llm():
    try:
        llm = Llama(
            model_path="em_german_7b_v01.Q8_0.gguf",  # Ensure this path is correct
            n_ctx=2048,
            verbose=False,
            n_batch=256,
            echo=False,
            temperature=0.2  # Lower temperature for more deterministic outputs
        )
        print("LLM erfolgreich initialisiert")
        return llm
    except Exception as e:
        print(f"Fehler bei der LLM-Initialisierung: {e}")
        print("Verwende Fallback-Modus ohne LLM")
        return None

# Define the workflow steps directly from the manual sections
workflow_steps = [
    "Montage der Komponenten",
    "Anschluss von Sensoren und externen Systemen",
    "Einschalten und Systemkonfiguration",
    "Anwendungsvorbereitung und Funktionstest"
]

def interpret_user_response(user_input):
    """Function that checks for confirmation words in German"""
    user_input = user_input.lower()
   
    # List of German confirmation words
    confirmation_words = [
        "fertig", "erledigt", "abgeschlossen", "ok", "ja",
        "gemacht", "geschafft", "bereit", "weiter"]
   
    # Check if any confirmation word is in the user input
    for word in confirmation_words:
        if word in user_input:
            return "yes"
   
    return "no"

# Main workflow
def main():
    print("CARDIOHELP-Einrichtungsassistent\n")
    print("Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.")
    print("--------------------------------------------------\n")
    
    # Process manual content and create vector store
    chunks = process_manual_content()
    vector_store = create_vector_store(chunks)
    
    # Initialize LLM - but we'll use pre-defined instructions as fallback
    llm = initialize_llm()
    
    current_step = 0
    
    while current_step < len(workflow_steps):
        step_name = workflow_steps[current_step]
        
        # Get relevant content from the manual for this step
        context = retrieve_context(vector_store, step_name)
        
        # Use pre-defined instructions if LLM fails or is not available
        instructions = ""
        
        if llm:
            try:
                # System prompt
                system_prompt = f"""
                Du bist ein hilfreicher CARDIOHELP-Einrichtungsassistent. Gib nur die konkreten Schritte an, 
                die der Benutzer ausführen muss. Verwende keine Einleitungen oder Fragen. 
                Antworte mit einen klaren und user-freundlichen, durchnummerierten Anleitung.
                Bitte die körperliche Bewebungen betonen.
                
                Schritt {current_step + 1}: {step_name}
                
                Relevante Informationen:
                {context}
                """
                
                # Generate a response
                response = llm(
                    system_prompt,
                    max_tokens=512,
                    temperature=0.2,
                    stop=["</s>", "<s>"],
                )
                
                instructions = response['choices'][0]['text'].strip()
                
                # Fallback to pre-defined instructions if LLM output seems inappropriate
                if "Bitte gib" in instructions or len(instructions) < 60:
                    print("LLM-Antwort scheint unvollständig - verwende vordefinierte Anweisungen")
                    instructions = STEP_INSTRUCTIONS[step_name]
                
            except Exception as e:
                print(f"Fehler bei der LLM-Antwort: {e}")
                instructions = STEP_INSTRUCTIONS[step_name]
        else:
            # Use pre-defined instructions if LLM is not available
            instructions = STEP_INSTRUCTIONS[step_name]
           
        # Display the instructions
        print(f"\n--- Schritt {current_step + 1}: {step_name} ---")
        print(instructions)
            
        # Get user confirmation to proceed
        #print("\nBitte bestätigen Sie, wenn Sie fertig sind (sagen Sie 'weiter', 'fertig', 'ok', etc.)")
        
        # For demonstration - in a real app, use: user_input = input("> ")
        user_input = random.choice(["weiter", "fertig", "ok", "ja", "erledigt", "gemacht"])
        print(f" User Antwort > {user_input}")
        
        # Check if user confirmed completion
        decision = interpret_user_response(user_input)
        if decision == "yes":
            current_step += 1
            if current_step < len(workflow_steps):
                print(f"\nWir gehen nun zu Schritt {current_step + 1} über.")
        else:
            print("Ich verstehe nicht. Bitte bestätigen Sie, wenn Sie fertig sind.")
    
    print("\nCARDIOHELP-Einrichtung abgeschlossen! Das Gerät ist nun einsatzbereit.")

if __name__ == "__main__":
    main()

CARDIOHELP-Einrichtungsassistent

Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.
--------------------------------------------------

Bedienungsanleitung in 11 Teile aufgeteilt
Vektordatenbank erfolgreich erstellt mit multilingual model


llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


LLM erfolgreich initialisiert
LLM-Antwort scheint unvollständig - verwende vordefinierte Anweisungen

--- Schritt 1: Montage der Komponenten ---

Bitte führen Sie die folgenden Schritte zur Montage der Komponenten aus:

1. Halterung für CARDIOHELP-i montieren:
   - Verwenden Sie die Halterung HKH 9102-M für 38mm Mastdurchmesser

2. Transportzubehör vorbereiten:
   - Bereiten Sie den Transporthalter HKH 8860 vor
   - Sichern Sie den CARDIOHELP Emergency Drive in der Transportbox

3. CARDIOHELP Emergency Drive positionieren:
   - Montieren Sie die Halterung für den Notantrieb (HKH 9101-R/M)

4. CARDIOHELP-i aufstellen:
   - Befestigen Sie das Gerät auf der Halterung
   - Öffnen Sie den Schutzbügel
   - Schließen Sie die Stromversorgung an
    
 User Antwort > weiter

Wir gehen nun zu Schritt 2 über.
LLM-Antwort scheint unvollständig - verwende vordefinierte Anweisungen

--- Schritt 2: Anschluss von Sensoren und externen Systemen ---

Bitte führen Sie die folgenden Schritte zum Anschluss 

KeyboardInterrupt: 

In [ ]:
import random
from llama_cpp import Llama
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# The compact manual content provided
MANUAL_CONTENT = """CARDIOHELP System-Vorbereitung in 4 Schritten
Schritt 1: Montage der Komponenten
Halterung für CARDIOHELP-i montieren: HKH 9102-M für 38mm Mastdurchmesser verwenden
Transportzubehör vorbereiten: Transporthalter HKH 8860, CARDIOHELP Emergency Drive in Transportbox sichern
CARDIOHELP Emergency Drive positionieren: Halterung für Notantrieb montieren (HKH 9101-R/M)
CARDIOHELP-i aufstellen: Gerät auf Halterung befestigen, Schutzbügel öffnen, Stromversorgung anschließen
Schritt 2: Anschluss von Sensoren und externen Systemen
Sensoren anschließen: Stecker anhand der roten Markierungen ausrichten
Externe Geräte verbinden: Datenaufzeichnungssystem über USB-Typ-B, externes Alarmsystem
Venösen Messkopf anschließen: An "Venöser Messkopf"-Anschluss und in Halterung platzieren
Sensorkabel für Einmalprodukt verbinden: Vor dem Einschalten anschließen für korrekte Erkennung
Schritt 3: Einschalten und Systemkonfiguration
Einschalten und Selbsttest: Venösen Messkopf vor Einschalten sichern, "Ein/Aus"-Taste drücken
Grundeinstellungen konfigurieren: Helligkeit, Lautstärke, Sprache, Zeit/Datum einstellen
Systeminformationen prüfen: Software-Version, Betriebsstunden, Batteriestatus kontrollieren
Key-User-Funktionen (passwortgeschützt): Alarmkonfiguration, Klinikkonfiguration speichern
Schritt 4: Anwendungsvorbereitung und Funktionstest
Perfusion vorbereiten: Einmalprodukt montieren, integrierte Sensorik anschließen
Erforderliche Sensoren positionieren: Fluss-/Blasensensor, Temperatursensoren, Drucksensoren
Funktionstests durchführen: Blasenüberwachung, Niveauüberwachung, Messkopf-Status prüfen
Kalibrierungen vornehmen: Nullkalibrierung für Drucksensoren, Fluss-Off-Set-Kalibrierung
Abschließende Überprüfung: Sicherheitssiegel, Bedienelemente, Sensorfunktionen, Notantrieb"""

# Function to process the manual text directly from content string
def process_manual_content():
    # Create a single Document object
    doc = Document(page_content=MANUAL_CONTENT, metadata={"source": "CARDIOHELP-Manual"})
    
    # Split the text into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,  # Smaller chunk size due to compact manual
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", ":", " ", ""]
    )
    chunks = text_splitter.split_documents([doc])
    
    print(f"Bedienungsanleitung in {len(chunks)} Teile aufgeteilt")
    return chunks


def create_vector_store(chunks):
    try:
        # Using multilingual model for better German support
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
            model_kwargs={"device": "cpu"},
        )
        
        # Create the vector store
        vector_store = FAISS.from_documents(chunks, embeddings)
        
        print("Vektordatenbank erfolgreich erstellt mit multilingual model")
        return vector_store
        
    except Exception as e:
        print(f"Fehler mit dem ersten Modell: {e}")
        print("Versuche alternatives Modell...")
        
        try:
            # Try another fallback model
            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2",
                model_kwargs={"device": "cpu"},
            )
            
            # Create the vector store
            vector_store = FAISS.from_documents(chunks, embeddings)
            
            print("Vektordatenbank erfolgreich erstellt mit alternativem Modell")
            return vector_store
            
        except Exception as e:
            print(f"Fehler mit dem zweiten Modell: {e}")
            print("Verwende lokale Embedding-Methode als Fallback")
            
            # If both models fail, use a basic TF-IDF vectorizer as fallback
            from sklearn.feature_extraction.text import TfidfVectorizer
            from sklearn.metrics.pairwise import cosine_similarity
            
            # Extract text from documents
            texts = [doc.page_content for doc in chunks]
            
            # Create TF-IDF vectors
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(texts)
            
            # Wrap in a simple class to mimic the FAISS interface
            class TfidfStore:
                def __init__(self, matrix, vectorizer, docs):
                    self.matrix = matrix
                    self.vectorizer = vectorizer
                    self.docs = docs
                
                def similarity_search(self, query, k=2):
                    # Transform query
                    query_vec = self.vectorizer.transform([query])
                    
                    # Calculate similarity
                    similarity = cosine_similarity(query_vec, self.matrix).flatten()
                    
                    # Get top k indices
                    indices = similarity.argsort()[:-k-1:-1]
                    
                    # Return top k documents
                    return [self.docs[i] for i in indices]
            
            tfidf_store = TfidfStore(tfidf_matrix, vectorizer, chunks)
            print("Vektordatenbank erfolgreich erstellt mit TF-IDF fallback")
            return tfidf_store

# Function to retrieve relevant context based on step instruction
def retrieve_context(vector_store, step_instruction, k=3):  # Increased k for more context
    # Retrieve the most similar chunks to the step instruction
    relevant_chunks = vector_store.similarity_search(step_instruction, k=k)
    
    # Extract the content and format as key points
    formatted_context = ""
    for i, chunk in enumerate(relevant_chunks):
        content = chunk.page_content.strip()
        # Format each line as a bullet point if it contains a colon
        lines = content.split('\n')
        for line in lines:
            if ':' in line:
                parts = line.split(':', 1)
                formatted_context += f"- {parts[0].strip()}: {parts[1].strip()}\n"
            else:
                formatted_context += f"- {line.strip()}\n"
    
    return formatted_context

# Structure the manual information into direct, clear instructions for each step
# These will be used only as a last resort if LLM can't generate good content
FALLBACK_INSTRUCTIONS = {
    "Montage der Komponenten": """
1. Halterung für CARDIOHELP-i montieren:
   • Verwenden Sie die Halterung HKH 9102-M (für 38mm Mastdurchmesser)
   • Befestigen Sie die Halterung sicher am Mast

2. Transportzubehör vorbereiten:
   • Stellen Sie den Transporthalter HKH 8860 bereit
   • Platzieren Sie den CARDIOHELP Emergency Drive in der Transportbox
   • Sichern Sie alle Komponenten gegen Verrutschen

3. CARDIOHELP Emergency Drive positionieren:
   • Identifizieren Sie die richtige Halterung für den Notantrieb (HKH 9101-R/M)
   • Montieren Sie diese an der vorgesehenen Position
   • Überprüfen Sie den festen Sitz der Halterung

4. CARDIOHELP-i aufstellen:
   • Setzen Sie das Gerät vorsichtig auf die montierte Halterung
   • Öffnen Sie den Schutzbügel vollständig
   • Verbinden Sie das Gerät mit der Stromversorgung
   • Kontrollieren Sie alle Anschlüsse auf korrekten Sitz
""",
    
    "Anschluss von Sensoren und externen Systemen": """
1. Sensoren korrekt anschließen:
   • Identifizieren Sie die roten Markierungen an allen Steckern
   • Richten Sie jeden Stecker anhand dieser Markierungen aus
   • Stellen Sie sicher, dass die Stecker einrasten
   
2. Externe Geräte verbinden:
   • Lokalisieren Sie den USB-Typ-B Anschluss am Gerät
   • Verbinden Sie das Datenaufzeichnungssystem mit diesem Anschluss
   • Schließen Sie das externe Alarmsystem an die dafür vorgesehene Buchse an
   • Prüfen Sie die Funktionsfähigkeit der Verbindungen

3. Venösen Messkopf anschließen:
   • Suchen Sie den mit "Venöser Messkopf" beschrifteten Anschluss
   • Verbinden Sie den Messkopf mit diesem Anschluss
   • Platzieren Sie den Messkopf sicher in seiner Halterung
   • Stellen Sie sicher, dass er stabil sitzt

4. Sensorkabel für Einmalprodukt verbinden:
   • WICHTIG: Diese Verbindung muss VOR dem Einschalten erfolgen!
   • Verbinden Sie alle Sensorkabel mit den Einmalprodukten
   • Achten Sie auf die korrekte Ausrichtung der Stecker
   • Überprüfen Sie den festen Sitz aller Verbindungen
""",
    
    "Einschalten und Systemkonfiguration": """
1. System einschalten und Selbsttest durchführen:
   • Stellen Sie sicher, dass der venöse Messkopf gesichert ist
   • Drücken Sie die "Ein/Aus"-Taste und halten Sie sie kurz gedrückt
   • Warten Sie, bis der automatische Selbsttest startet
   • Beobachten Sie den Selbsttest und achten Sie auf Fehlermeldungen

2. Grundeinstellungen nach Bedarf konfigurieren:
   • Helligkeit: Passen Sie die Bildschirmhelligkeit an die Umgebung an
   • Lautstärke: Stellen Sie eine angemessene Alarmlautstärke ein
   • Sprache: Wählen Sie Ihre bevorzugte Sprache für die Benutzeroberfläche
   • Zeit/Datum: Aktualisieren Sie die Zeit- und Datumseinstellungen

3. Wichtige Systeminformationen überprüfen:
   • Kontrollieren Sie die aktuelle Software-Version
   • Überprüfen Sie die Gesamtbetriebsstunden des Geräts
   • Kontrollieren Sie den Batteriestatus und laden Sie bei Bedarf
   • Dokumentieren Sie diese Informationen bei Bedarf

4. Key-User-Funktionen (nur mit Passwort zugänglich):
   • Greifen Sie auf die erweiterten Funktionen zu (Passwort erforderlich)
   • Konfigurieren Sie die Alarmeinstellungen nach klinischen Anforderungen
   • Speichern Sie die spezifische Klinikkonfiguration
   • Führen Sie weitere passwortgeschützte Einstellungen durch
""",
    
    "Anwendungsvorbereitung und Funktionstest": """
1. Perfusion gewissenhaft vorbereiten:
   • Nehmen Sie das Einmalprodukt aus der sterilen Verpackung
   • Montieren Sie es gemäß den Herstelleranweisungen
   • Schließen Sie alle integrierten Sensoren sorgfältig an
   • Überprüfen Sie auf korrekte Platzierung und Anschluss

2. Erforderliche Sensoren korrekt positionieren:
   • Platzieren Sie den Fluss-/Blasensensor an der vorgesehenen Stelle
   • Bringen Sie alle Temperatursensoren an den markierten Positionen an
   • Positionieren Sie die Drucksensoren gemäß Anleitung
   • Überprüfen Sie den korrekten Sitz aller Sensoren

3. Umfassende Funktionstests durchführen:
   • Testen Sie die Blasenüberwachung auf korrekte Funktion
   • Überprüfen Sie die Niveauüberwachung
   • Kontrollieren Sie den Status aller Messkopfkomponenten
   • Achten Sie auf Warnmeldungen oder Funktionsstörungen

4. Notwendige Kalibrierungen vornehmen:
   • Führen Sie die Nullkalibrierung für alle Drucksensoren durch
   • Kalibrieren Sie den Fluss-Off-Set nach Anleitung
   • Dokumentieren Sie alle durchgeführten Kalibrierungen
   • Wiederholen Sie Kalibrierungen bei Bedarf

5. Abschließende Sicherheitsüberprüfung:
   • Kontrollieren Sie alle Sicherheitssiegel auf Unversehrtheit
   • Testen Sie sämtliche Bedienelemente auf korrekte Funktion
   • Überprüfen Sie alle Sensorfunktionen ein letztes Mal
   • Führen Sie einen Test des Notantriebs durch
   • Stellen Sie sicher, dass das System einsatzbereit ist
"""
}

# Collection of different prompts for LLM to generate varied responses
# These templates will help get better, more varied responses from the LLM
PROMPT_TEMPLATES = [
    # Template 1: Direct instruction format
    """
Du bist ein Medizintechnik-Experte für das CARDIOHELP-System. Deine Aufgabe ist es,
klare Anweisungen für den folgenden Schritt zu geben:

Schritt: {step_name}

Verwende diese Informationen aus dem Handbuch:
{context}

Formatiere deine Antwort als nummerierte Liste mit Hauptpunkten und Unterpunkten.
Sei präzise und klar. Verwende eine freundliche aber professionelle Sprache.
Konzentriere dich NUR auf die praktischen Schritte, die der Benutzer ausführen muss.
""",

    # Template 2: Procedural format
    """
Als CARDIOHELP-Systemtrainer sollst du dem medizinischen Personal folgende Aufgabe erklären:

AUFGABE: {step_name}

REFERENZINFORMATIONEN:
{context}

Erstelle eine detaillierte, schrittweise Anleitung mit Hauptschritten (nummeriert) und 
wichtigen Details als Unterpunkte. Verwende klare, direkte Sprache. 
Beginne jeden Schritt mit einem Aktionsverb.
""",

    # Template 3: Expert walkthrough
    """
Du bist ein erfahrener Anwendungsspezialist für CARDIOHELP-Systeme in Kliniken.
Ein neuer Mitarbeiter bittet dich um Hilfe bei:

"{step_name}"

Basierend auf den Herstellerinformationen:
{context}

Erkläre den Prozess in klaren, fortlaufenden Schritten. Nummeriere jeden Hauptschritt
und füge bei Bedarf Unterpunkte hinzu. Verwende präzise Fachsprache, aber bleibe verständlich.
Achte besonders auf die Reihenfolge und sicherheitsrelevante Details.
"""
]

# Initialize the model with appropriate settings
def initialize_llm():
    try:
        llm = Llama(
            model_path="em_german_7b_v01.Q8_0.gguf",  # Ensure this path is correct
            n_ctx=2048,
            verbose=False,
            n_batch=256,
            echo=False,
            temperature=0.45  # Higher temperature for more variety
        )
        print("LLM erfolgreich initialisiert")
        return llm
    except Exception as e:
        print(f"Fehler bei der LLM-Initialisierung: {e}")
        print("Verwende Fallback-Modus ohne LLM")
        return None

# Define the workflow steps directly from the manual sections
workflow_steps = [
    "Montage der Komponenten",
    "Anschluss von Sensoren und externen Systemen",
    "Einschalten und Systemkonfiguration",
    "Anwendungsvorbereitung und Funktionstest"
]

# Varied confirmation phrases to make the interaction more natural
CONFIRMATION_RESPONSES = [
    "Sehr gut! Schritt {step_num} ist abgeschlossen.",
    "Ausgezeichnet, wir können mit dem nächsten Schritt fortfahren.",
    "Perfekt erledigt. Gehen wir weiter zu Schritt {step_num}.",
    "Danke für die Bestätigung. Wir setzen nun mit Schritt {step_num} fort.",
    "Gut gemacht! Bereit für den nächsten Schritt?",
    "Prima, der aktuelle Schritt wurde erfolgreich abgeschlossen."
]

def interpret_user_response(user_input):
    """Function that checks for confirmation words in German"""
    user_input = user_input.lower()
   
    # List of German confirmation words
    confirmation_words = [
        "fertig", "erledigt", "abgeschlossen", "ok", "ja",
        "gemacht", "geschafft", "bereit", "weiter", "klar"
    ]
   
    # Check if any confirmation word is in the user input
    for word in confirmation_words:
        if word in user_input:
            return "yes"
   
    return "no"

def validate_llm_response(response_text, step_name):
    """Check if LLM response is valid and useful"""
    # Some basic validation checks
    
    # Check if response is too short
    if len(response_text) < 100:
        return False
        
    # Check for common error patterns
    error_patterns = [
        "Bitte gib", "Kann ich", "Hier sind", "Ich verstehe",
        "Ich bin ein", "Als Assistent", "Als KI", "Ich helfe"
    ]
    
    for pattern in error_patterns:
        if pattern in response_text:
            return False
    
    # Check if the response contains numbered steps (1., 2., etc.)
    has_numbered_steps = any(f"{i}." in response_text for i in range(1, 10))
    if not has_numbered_steps:
        return False
        
    # Check if response is relevant to the current step
    step_keywords = {
        "Montage der Komponenten": ["halterung", "montieren", "gerät", "befestigen"],
        "Anschluss von Sensoren": ["sensor", "anschließen", "stecker", "verbinden"],
        "Einschalten und Systemkonfiguration": ["einschalten", "konfigurieren", "test", "taste"],
        "Anwendungsvorbereitung und Funktionstest": ["kalibrierung", "sensor", "test", "überprüfung"]
    }
    
    # Get keywords for the current step
    if step_name in step_keywords:
        keywords = step_keywords[step_name]
        # Check if at least half of the keywords are in the response
        keyword_matches = sum(1 for kw in keywords if kw.lower() in response_text.lower())
        if keyword_matches < len(keywords) // 2:
            return False
    
    return True

# Main workflow
def main():
    print("CARDIOHELP-Einrichtungsassistent\n")
    print("Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.")
    print("--------------------------------------------------\n")
    
    # Process manual content and create vector store
    chunks = process_manual_content()
    vector_store = create_vector_store(chunks)
    
    # Initialize LLM - but we'll use pre-defined instructions as fallback
    llm = initialize_llm()
    
    current_step = 0
    
    while current_step < len(workflow_steps):
        step_name = workflow_steps[current_step]
        
        # Get relevant content from the manual for this step
        context = retrieve_context(vector_store, step_name)
        
        # Use pre-defined instructions if LLM fails or is not available
        instructions = ""
        
        if llm:
            try:
                # Choose a random prompt template for variety
                prompt_template = random.choice(PROMPT_TEMPLATES)
                
                # Fill in the template
                system_prompt = prompt_template.format(
                    step_name=step_name,
                    context=context
                )
                
                # Generate a response with higher temperature for more diversity
                response = llm(
                    system_prompt,
                    max_tokens=800,  # Increased for more detailed responses
                    temperature=0.5 + (current_step * 0.05),  # Slightly increase temperature for each step
                    stop=["</s>", "<s>"],
                )
                
                instructions = response['choices'][0]['text'].strip()
                
                # Validate the LLM response
                if not validate_llm_response(instructions, step_name):
                    print("LLM-Antwort scheint nicht optimal - verwende alternative Anweisungen")
                    # Use fallback instructions for this step
                    instructions = FALLBACK_INSTRUCTIONS[step_name]
                
            except Exception as e:
                print(f"Fehler bei der LLM-Antwort: {e}")
                instructions = FALLBACK_INSTRUCTIONS[step_name]
        else:
            # Use pre-defined instructions if LLM is not available
            instructions = FALLBACK_INSTRUCTIONS[step_name]
           
        # Display the instructions
        print(f"\n--- Schritt {current_step + 1}: {step_name} ---")
        print(instructions)
            
        # Get user confirmation to proceed
        #print("\nBitte bestätigen Sie, wenn Sie fertig sind (sagen Sie 'weiter', 'fertig', 'ok', etc.)")
        
        # For demonstration - in a real app, use: user_input = input("> ")
        user_input = random.choice(["weiter", "fertig", "ok", "ja", "erledigt", "klar"])
        print(f" User Antwort > {user_input}")
        
        # Check if user confirmed completion
        decision = interpret_user_response(user_input)
        if decision == "yes":
            current_step += 1
            if current_step < len(workflow_steps):
                # Choose a random confirmation response for variety
                confirmation = random.choice(CONFIRMATION_RESPONSES).format(step_num=current_step + 1)
                print(f"\n{confirmation}")
            else:
                print("\nHerzlichen Glückwunsch! Sie haben alle Schritte erfolgreich abgeschlossen.")
        else:
            print("Ich verstehe nicht. Bitte bestätigen Sie, wenn Sie fertig sind.")
    
    print("\nCARDIOHELP-Einrichtung abgeschlossen! Das Gerät ist nun einsatzbereit.")

if __name__ == "__main__":
    main()

In [ ]:
import random
from llama_cpp import Llama
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# The compact manual content provided
MANUAL_CONTENT = """CARDIOHELP System-Vorbereitung in 4 Schritten
Schritt 1: Montage der Komponenten
Halterung für CARDIOHELP-i montieren: HKH 9102-M für 38mm Mastdurchmesser verwenden
Transportzubehör vorbereiten: Transporthalter HKH 8860, CARDIOHELP Emergency Drive in Transportbox sichern
CARDIOHELP Emergency Drive positionieren: Halterung für Notantrieb montieren (HKH 9101-R/M)
CARDIOHELP-i aufstellen: Gerät auf Halterung befestigen, Schutzbügel öffnen, Stromversorgung anschließen
Schritt 2: Anschluss von Sensoren und externen Systemen
Sensoren anschließen: Stecker anhand der roten Markierungen ausrichten
Externe Geräte verbinden: Datenaufzeichnungssystem über USB-Typ-B, externes Alarmsystem
Venösen Messkopf anschließen: An "Venöser Messkopf"-Anschluss und in Halterung platzieren
Sensorkabel für Einmalprodukt verbinden: Vor dem Einschalten anschließen für korrekte Erkennung
Schritt 3: Einschalten und Systemkonfiguration
Einschalten und Selbsttest: Venösen Messkopf vor Einschalten sichern, "Ein/Aus"-Taste drücken
Grundeinstellungen konfigurieren: Helligkeit, Lautstärke, Sprache, Zeit/Datum einstellen
Systeminformationen prüfen: Software-Version, Betriebsstunden, Batteriestatus kontrollieren
Key-User-Funktionen (passwortgeschützt): Alarmkonfiguration, Klinikkonfiguration speichern
Schritt 4: Anwendungsvorbereitung und Funktionstest
Perfusion vorbereiten: Einmalprodukt montieren, integrierte Sensorik anschließen
Erforderliche Sensoren positionieren: Fluss-/Blasensensor, Temperatursensoren, Drucksensoren
Funktionstests durchführen: Blasenüberwachung, Niveauüberwachung, Messkopf-Status prüfen
Kalibrierungen vornehmen: Nullkalibrierung für Drucksensoren, Fluss-Off-Set-Kalibrierung
Abschließende Überprüfung: Sicherheitssiegel, Bedienelemente, Sensorfunktionen, Notantrieb"""

# Function to process the manual text directly from content string
def process_manual_content():
    # Create a single Document object
    doc = Document(page_content=MANUAL_CONTENT, metadata={"source": "CARDIOHELP-Manual"})
    
    # Split the text into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,  # Smaller chunk size due to compact manual
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", ":", " ", ""]
    )
    chunks = text_splitter.split_documents([doc])
    
    print(f"Bedienungsanleitung in {len(chunks)} Teile aufgeteilt")
    return chunks


def create_vector_store(chunks):
    try:
        # Using multilingual model for better German support
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
            model_kwargs={"device": "cpu"},
        )
        
        # Create the vector store
        vector_store = FAISS.from_documents(chunks, embeddings)
        
        print("Vektordatenbank erfolgreich erstellt mit multilingual model")
        return vector_store
        
    except Exception as e:
        print(f"Fehler mit dem ersten Modell: {e}")
        print("Versuche alternatives Modell...")
        
        try:
            # Try another fallback model
            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2",
                model_kwargs={"device": "cpu"},
            )
            
            # Create the vector store
            vector_store = FAISS.from_documents(chunks, embeddings)
            
            print("Vektordatenbank erfolgreich erstellt mit alternativem Modell")
            return vector_store
            
        except Exception as e:
            print(f"Fehler mit dem zweiten Modell: {e}")
            print("Verwende lokale Embedding-Methode als Fallback")
            
            # If both models fail, use a basic TF-IDF vectorizer as fallback
            from sklearn.feature_extraction.text import TfidfVectorizer
            from sklearn.metrics.pairwise import cosine_similarity
            
            # Extract text from documents
            texts = [doc.page_content for doc in chunks]
            
            # Create TF-IDF vectors
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(texts)
            
            # Wrap in a simple class to mimic the FAISS interface
            class TfidfStore:
                def __init__(self, matrix, vectorizer, docs):
                    self.matrix = matrix
                    self.vectorizer = vectorizer
                    self.docs = docs
                
                def similarity_search(self, query, k=2):
                    # Transform query
                    query_vec = self.vectorizer.transform([query])
                    
                    # Calculate similarity
                    similarity = cosine_similarity(query_vec, self.matrix).flatten()
                    
                    # Get top k indices
                    indices = similarity.argsort()[:-k-1:-1]
                    
                    # Return top k documents
                    return [self.docs[i] for i in indices]
            
            tfidf_store = TfidfStore(tfidf_matrix, vectorizer, chunks)
            print("Vektordatenbank erfolgreich erstellt mit TF-IDF fallback")
            return tfidf_store

# Function to retrieve relevant context based on step instruction
def retrieve_context(vector_store, step_instruction, k=2):
    # Retrieve the most similar chunks to the step instruction
    relevant_chunks = vector_store.similarity_search(step_instruction, k=k)
    
    # Concatenate the content of the chunks
    context = "\n\n".join([doc.page_content for doc in relevant_chunks])
    
    return context

# Improved step instructions with more variety and natural language
STEP_INSTRUCTIONS_VARIANTS = {
    "Montage der Komponenten": [
        """
Führen Sie bitte folgende Montageschritte aus:

1. CARDIOHELP-i Halterung anbringen:
   - HKH 9102-M Halterung (für 38mm Mastdurchmesser) befestigen
   
2. Transport vorbereiten:
   - HKH 8860 Transporthalter bereitstellen
   - Emergency Drive sicher in der Transportbox verstauen
   
3. Notantrieb installieren:
   - HKH 9101-R/M Halterung für den Emergency Drive montieren
   
4. Hauptgerät installieren:
   - CARDIOHELP-i auf der Halterung sicher befestigen
   - Schutzbügel nach oben klappen
   - Stromkabel anschließen und Versorgung sicherstellen
        """,
        
        """
Montieren Sie das CARDIOHELP-System wie folgt:

1. Halterungsmontage:
   - Nehmen Sie die HKH 9102-M Halterung (38mm)
   - Befestigen Sie diese am vorgesehenen Mast
   
2. Transportsystem vorbereiten:
   - HKH 8860 Transporthalter positionieren
   - CARDIOHELP Emergency Drive in der Transportbox verstauen
   
3. Notantrieb-Vorbereitung:
   - Bringen Sie die HKH 9101-R/M Halterung an
   - Bereiten Sie den Notantrieb für die Montage vor
   
4. Hauptsystem aufbauen:
   - CARDIOHELP-i vorsichtig auf der Halterung platzieren
   - Entriegeln Sie den Schutzbügel und öffnen Sie ihn
   - Verbinden Sie das Gerät mit der Stromversorgung
        """
    ],
    
    "Anschluss von Sensoren und externen Systemen": [
        """
Schließen Sie Sensoren und externe Systeme wie folgt an:

1. Sensoren verbinden:
   - Achten Sie auf die roten Markierungen bei allen Steckern
   - Führen Sie die Stecker gerade ein, ohne zu verkanten
   
2. Externe Komponenten anschließen:
   - Datenaufzeichnungssystem mit dem USB-Typ-B Port verbinden
   - Externes Alarmsystem gemäß Kennzeichnung anschließen
   
3. Venösen Messkopf montieren:
   - Stecken Sie den Messkopf in den gekennzeichneten Anschluss
   - Setzen Sie ihn anschließend in die dafür vorgesehene Halterung
   
4. Einmalprodukt-Sensoren verbinden:
   - WICHTIG: Alle Sensorkabel VOR dem Einschalten anschließen
   - Auf korrekte Ausrichtung für die automatische Erkennung achten
        """,
        
        """
Verbinden Sie nun alle Sensoren und Systeme:

1. Sensoranschlüsse herstellen:
   - Orientieren Sie sich an den roten Markierungen auf allen Steckverbindungen
   - Stellen Sie sicher, dass alle Anschlüsse fest sitzen
   
2. Verbindung zu externen Geräten:
   - Schließen Sie das Datenaufzeichnungssystem über den USB-B-Port an
   - Verbinden Sie das externe Alarmierungssystem entsprechend den Markierungen
   
3. Venösen Messkopf einrichten:
   - Stecken Sie den Kopf in den "Venöser Messkopf"-Port
   - Rasten Sie ihn danach in der Halterung ein
   
4. Vorbereitung der Einmalprodukt-Sensorik:
   - Sensorkabel müssen unbedingt VOR dem Systemstart angeschlossen sein
   - Beachten Sie die korrekte Orientierung für optimale Funktionalität
        """
    ],
    
    "Einschalten und Systemkonfiguration": [
        """
Schalten Sie das System ein und konfigurieren Sie es:

1. Einschaltvorgang:
   - Überprüfen Sie, dass der venöse Messkopf korrekt positioniert ist
   - Drücken Sie die "Ein/Aus"-Taste und warten Sie auf den Selbsttest
   
2. Displayeinstellungen anpassen:
   - Stellen Sie die Bildschirmhelligkeit auf einen angenehmen Wert
   - Passen Sie die Systemlautstärke an die Umgebung an
   - Wählen Sie Ihre bevorzugte Sprache aus
   - Aktualisieren Sie Datum und Uhrzeit
   
3. Systemdiagnose:
   - Überprüfen Sie die installierte Software-Version
   - Kontrollieren Sie die angezeigten Betriebsstunden
   - Prüfen Sie den Ladezustand der Batterien
   
4. Erweiterte Einstellungen (mit Passwort):
   - Konfigurieren Sie die Alarmparameter nach Klinikstandard
   - Speichern Sie die personalisierten Einstellungen
        """,
        
        """
System starten und einrichten:

1. System aktivieren:
   - Stellen Sie sicher, dass der venöse Messkopf gesichert ist
   - Betätigen Sie den Einschaltknopf und beobachten Sie den Selbsttest
   
2. Basiskonfiguration durchführen:
   - Passen Sie die Displayhelligkeit an die Lichtverhältnisse an
   - Regulieren Sie die Alarmtonlautstärke
   - Stellen Sie die gewünschte Benutzersprache ein
   - Konfigurieren Sie die korrekte Zeit und das Datum
   
3. Systeminformationen überprüfen:
   - Notieren Sie die aktuelle Software-Version
   - Überprüfen Sie die aufgezeichneten Betriebsstunden
   - Kontrollieren Sie den Batteriestatus aller Komponenten
   
4. Administrator-Funktionen (Passworteingabe erforderlich):
   - Passen Sie die Alarmgrenzwerte an
   - Speichern Sie die klinikspezifischen Konfigurationen im System
        """
    ],
    
    "Anwendungsvorbereitung und Funktionstest": [
        """
Bereiten Sie die Anwendung vor und führen Sie Funktionstests durch:

1. Perfusionssystem vorbereiten:
   - Installieren Sie das Einmalprodukt entsprechend der Markierungen
   - Verbinden Sie alle integrierten Sensoren mit den Anschlüssen
   
2. Sensorkonfiguration:
   - Positionieren Sie den Fluss- und Blasensensor präzise
   - Bringen Sie die Temperatursensoren an den Messpunkten an
   - Installieren Sie die Drucksensoren an den vorgesehenen Stellen
   
3. Systemtests durchführen:
   - Testen Sie die Funktion der Blasenerkennung
   - Überprüfen Sie die Niveauüberwachung auf korrekte Reaktion
   - Kontrollieren Sie den Status aller Messköpfe
   
4. Sensorkalibrierung:
   - Führen Sie die Nullpunkt-Kalibrierung für alle Drucksensoren durch
   - Kalibrieren Sie den Fluss-Offset für präzise Messungen
   
5. Abschließende Sicherheitsprüfung:
   - Überprüfen Sie alle Sicherheitssiegel auf Unversehrtheit
   - Testen Sie die Funktion aller Bedienelemente
   - Kontrollieren Sie sämtliche Sensorfunktionen
   - Führen Sie einen Testlauf des Notantriebs durch
        """,
        
        """
Führen Sie die finalen Vorbereitungen und Tests durch:

1. Perfusionseinheit einrichten:
   - Montieren Sie das Einmalprodukt nach Herstellerangaben
   - Verbinden Sie die Sensorik mit den entsprechenden Ports
   
2. Sensorpositionierung:
   - Bringen Sie den kombinierten Fluss-/Blasensensor an
   - Platzieren Sie alle Temperatursensoren an den Messstellen
   - Installieren Sie die Drucksensoren gemäß Anleitung
   
3. Funktionsüberprüfung:
   - Testen Sie die Funktionalität der Blasendetektion
   - Prüfen Sie die Reaktion der Niveauüberwachung
   - Verifizieren Sie den Status des Messkopfsystems
   
4. Systemkalibrierung:
   - Führen Sie für alle Drucksensoren eine Nullkalibrierung durch
   - Justieren Sie den Fluss-Off-Set nach Herstellervorgaben
   
5. Finaler Sicherheitscheck:
   - Inspizieren Sie alle Sicherheitssiegel
   - Überprüfen Sie die Funktion sämtlicher Bedienelemente
   - Testen Sie alle verbundenen Sensoren
   - Verifizieren Sie die Funktionalität des Notantriebs
        """
    ]
}

# Initialize the model with appropriate settings
def initialize_llm():
    try:
        llm = Llama(
            model_path="em_german_7b_v01.Q8_0.gguf",  # Ensure this path is correct
            n_ctx=2048,
            verbose=False,
            n_batch=256,
            echo=False,
            temperature=0.7  # Increased temperature for more variety
        )
        print("LLM erfolgreich initialisiert")
        return llm
    except Exception as e:
        print(f"Fehler bei der LLM-Initialisierung: {e}")
        print("Verwende Fallback-Modus ohne LLM")
        return None

# Define the workflow steps directly from the manual sections
workflow_steps = [
    "Montage der Komponenten",
    "Anschluss von Sensoren und externen Systemen",
    "Einschalten und Systemkonfiguration",
    "Anwendungsvorbereitung und Funktionstest"
]

# Improved variety for user response interpretation
def interpret_user_response(user_input):
    """Function that checks for confirmation words in German with more sophistication"""
    user_input = user_input.lower().strip()
   
    # Comprehensive list of German confirmation words/phrases
    confirmation_words = [
        "fertig", "erledigt", "abgeschlossen", "ok", "ja", "jawohl",
        "gemacht", "geschafft", "bereit", "weiter", "klar", "verstanden",
        "in ordnung", "passt", "kann weitergehen", "nächster schritt",
        "alles klar", "geht weiter", "bin soweit", "bin bereit", 
        "nächster", "okay", "check", "gut", "geht", "läuft", "weiter geht's"
    ]
   
    # Check if any confirmation word is in the user input
    for word in confirmation_words:
        if word in user_input:
            return "yes"
   
    return "no"

# Collection of different motivational/transition messages for variety
TRANSITION_MESSAGES = [
    "Sehr gut! Gehen wir zum nächsten Schritt über.",
    "Ausgezeichnet! Der nächste Schritt wartet auf Sie.",
    "Gut gemacht! Fahren wir mit dem nächsten Schritt fort.",
    "Prima! Jetzt zum nächsten Schritt.",
    "Perfekt! Weiter geht's mit dem nächsten Punkt.",
    "Hervorragend! Nun zum nächsten Arbeitsschritt.",
    "Das haben Sie gut gemeistert! Kommen wir zum nächsten Schritt.",
    "Verstanden! Fahren wir mit dem nächsten Schritt fort.",
    "Alles klar! Gehen wir weiter zum nächsten Schritt."
]

# Collection of error messages for variety
ERROR_MESSAGES = [
    "Entschuldigung, ich habe Ihre Antwort nicht verstanden. Bitte bestätigen Sie, wenn Sie fertig sind.",
    "Ich konnte nicht erkennen, ob Sie den Schritt abgeschlossen haben. Bitte sagen Sie 'fertig', 'weiter' oder ähnliches.",
    "Ihre Antwort war unklar. Bitte teilen Sie mir mit, wenn Sie bereit für den nächsten Schritt sind.",
    "Könnten Sie bitte bestätigen, dass Sie den Schritt abgeschlossen haben?",
    "Ich bin mir nicht sicher, ob Sie fortfahren möchten. Bitte bestätigen Sie mit 'weiter', 'fertig' oder 'ok'."
]

# Collection of completion messages for variety
COMPLETION_MESSAGES = [
    "CARDIOHELP-Einrichtung erfolgreich abgeschlossen! Das Gerät ist nun einsatzbereit.",
    "Hervorragend! Die CARDIOHELP-Einrichtung ist vollständig abgeschlossen. Das System ist jetzt betriebsbereit.",
    "Alle Einrichtungsschritte wurden erfolgreich durchgeführt! Das CARDIOHELP-System kann nun verwendet werden.",
    "Glückwunsch! Die CARDIOHELP-Systemeinrichtung wurde erfolgreich abgeschlossen. Das Gerät ist einsatzbereit.",
    "Die Einrichtung des CARDIOHELP-Systems ist nun vollständig abgeschlossen. Das Gerät steht zur Verfügung."
]

# Main workflow with improved variety
def main():
    print("CARDIOHELP-Einrichtungsassistent\n")
    print("Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.")
    print("--------------------------------------------------\n")
    
    # Process manual content and create vector store
    chunks = process_manual_content()
    vector_store = create_vector_store(chunks)
    
    # Initialize LLM
    llm = initialize_llm()
    
    current_step = 0
    
    while current_step < len(workflow_steps):
        step_name = workflow_steps[current_step]
        
        # Get relevant content from the manual for this step
        context = retrieve_context(vector_store, step_name)
        
        # Default to a random variant from our predefined instructions
        instructions = random.choice(STEP_INSTRUCTIONS_VARIANTS[step_name])
        
        if llm:
            try:
                # Improved prompt engineering with less context that might confuse the LLM
                # and clearer instructions on what to generate
                system_prompt = f"""
                Du bist ein hilfreicher medizinischer Assistent, der bei der Einrichtung des CARDIOHELP-Systems unterstützt.
                
                Erstelle eine klare, präzise Anleitung für:
                Schritt {current_step + 1}: {step_name}
                
                Wichtig:
                1. Formuliere komplette, praktische Anweisungen in natürlichem Deutsch
                2. Verwende nummerierte Listen für die Teilschritte
                3. Beginne DIREKT mit den Anweisungen ohne Einleitungstexte
                4. Verwende keine Füllwörter oder allgemeine Floskeln
                
                Relevante Information:
                {context}
                """
                
                # Generate a response
                response = llm(
                    system_prompt,
                    max_tokens=700,
                    temperature=0.7,  # Higher temperature for variety
                    stop=["</s>", "<s>", "Wichtig:", "Relevante Information:"],
                )
                
                llm_output = response['choices'][0]['text'].strip()
                
                # Improved validation to detect low-quality outputs
                low_quality_indicators = [
                    "Bitte gib", "Ich bin", "Ich werde", "Als Ihr", "Gerne erstelle",
                    "Hier ist", "Verstanden", "Ich kann", "Natürlich"
                ]
                
                # Check if the output is too short, contains meta-commentary, or is low quality
                if (len(llm_output) < 100 or
                    any(indicator in llm_output for indicator in low_quality_indicators) or
                    "Schritt" in llm_output[:20]):  # LLM repeating the step intro
                    
                    print("LLM-Antwort entspricht nicht den Qualitätsanforderungen - verwende vordefinierte Anweisung")
                else:
                    instructions = llm_output
                
            except Exception as e:
                print(f"Fehler bei der LLM-Antwort: {e}")
                # In case of error, fall back to our predefined variants
        
        # Display the instructions with better formatting
        print(f"\n--- Schritt {current_step + 1}: {step_name} ---")
        print(instructions.strip())
            
        # For demonstration - in a real app, use: user_input = input("> ")
        user_input = random.choice(["weiter", "fertig", "ok", "ja", "erledigt", "gemacht"])
        print(f"\n> {user_input}")
        
        # Check if user confirmed completion
        decision = interpret_user_response(user_input)
        if decision == "yes":
            current_step += 1
            if current_step < len(workflow_steps):
                # Use variable transition messages for more natural feeling
                print(f"\n{random.choice(TRANSITION_MESSAGES)}")
            else:
                # Use variable completion messages
                print(f"\n{random.choice(COMPLETION_MESSAGES)}")
        else:
            # Use variable error messages
            print(random.choice(ERROR_MESSAGES))
    
if __name__ == "__main__":
    main()

In [1]:
import random
from llama_cpp import Llama
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# The compact manual content provided
MANUAL_CONTENT = """CARDIOHELP System-Vorbereitung in 4 Schritten
Schritt 1: Montage der Komponenten
Halterung für CARDIOHELP-i montieren: HKH 9102-M für 38mm Mastdurchmesser verwenden
Transportzubehör vorbereiten: Transporthalter HKH 8860, CARDIOHELP Emergency Drive in Transportbox sichern
CARDIOHELP Emergency Drive positionieren: Halterung für Notantrieb montieren (HKH 9101-R/M)
CARDIOHELP-i aufstellen: Gerät auf Halterung befestigen, Schutzbügel öffnen, Stromversorgung anschließen
Schritt 2: Anschluss von Sensoren und externen Systemen
Sensoren anschließen: Stecker anhand der roten Markierungen ausrichten
Externe Geräte verbinden: Datenaufzeichnungssystem über USB-Typ-B, externes Alarmsystem
Venösen Messkopf anschließen: An "Venöser Messkopf"-Anschluss und in Halterung platzieren
Sensorkabel für Einmalprodukt verbinden: Vor dem Einschalten anschließen für korrekte Erkennung
Schritt 3: Einschalten und Systemkonfiguration
Einschalten und Selbsttest: Venösen Messkopf vor Einschalten sichern, "Ein/Aus"-Taste drücken
Grundeinstellungen konfigurieren: Helligkeit, Lautstärke, Sprache, Zeit/Datum einstellen
Systeminformationen prüfen: Software-Version, Betriebsstunden, Batteriestatus kontrollieren
Key-User-Funktionen (passwortgeschützt): Alarmkonfiguration, Klinikkonfiguration speichern
Schritt 4: Anwendungsvorbereitung und Funktionstest
Perfusion vorbereiten: Einmalprodukt montieren, integrierte Sensorik anschließen
Erforderliche Sensoren positionieren: Fluss-/Blasensensor, Temperatursensoren, Drucksensoren
Funktionstests durchführen: Blasenüberwachung, Niveauüberwachung, Messkopf-Status prüfen
Kalibrierungen vornehmen: Nullkalibrierung für Drucksensoren, Fluss-Off-Set-Kalibrierung
Abschließende Überprüfung: Sicherheitssiegel, Bedienelemente, Sensorfunktionen, Notantrieb"""

# Function to process the manual text directly from content string
def process_manual_content():
    # Create a single Document object
    doc = Document(page_content=MANUAL_CONTENT, metadata={"source": "CARDIOHELP-Manual"})
    
    # Split the text into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,  # Smaller chunk size due to compact manual
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", ":", " ", ""]
    )
    chunks = text_splitter.split_documents([doc])
    
    print(f"Bedienungsanleitung in {len(chunks)} Teile aufgeteilt")
    return chunks


def create_vector_store(chunks):
    try:
        # Using multilingual model for better German support
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
            model_kwargs={"device": "cpu"},
        )
        
        # Create the vector store
        vector_store = FAISS.from_documents(chunks, embeddings)
        
        print("Vektordatenbank erfolgreich erstellt mit multilingual model")
        return vector_store
        
    except Exception as e:
        print(f"Fehler mit dem ersten Modell: {e}")
        print("Versuche alternatives Modell...")
        
        try:
            # Try another fallback model
            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2",
                model_kwargs={"device": "cpu"},
            )
            
            # Create the vector store
            vector_store = FAISS.from_documents(chunks, embeddings)
            
            print("Vektordatenbank erfolgreich erstellt mit alternativem Modell")
            return vector_store
            
        except Exception as e:
            print(f"Fehler mit dem zweiten Modell: {e}")
            print("Verwende lokale Embedding-Methode als Fallback")
            
            # If both models fail, use a basic TF-IDF vectorizer as fallback
            from sklearn.feature_extraction.text import TfidfVectorizer
            from sklearn.metrics.pairwise import cosine_similarity
            
            # Extract text from documents
            texts = [doc.page_content for doc in chunks]
            
            # Create TF-IDF vectors
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(texts)
            
            # Wrap in a simple class to mimic the FAISS interface
            class TfidfStore:
                def __init__(self, matrix, vectorizer, docs):
                    self.matrix = matrix
                    self.vectorizer = vectorizer
                    self.docs = docs
                
                def similarity_search(self, query, k=2):
                    # Transform query
                    query_vec = self.vectorizer.transform([query])
                    
                    # Calculate similarity
                    similarity = cosine_similarity(query_vec, self.matrix).flatten()
                    
                    # Get top k indices
                    indices = similarity.argsort()[:-k-1:-1]
                    
                    # Return top k documents
                    return [self.docs[i] for i in indices]
            
            tfidf_store = TfidfStore(tfidf_matrix, vectorizer, chunks)
            print("Vektordatenbank erfolgreich erstellt mit TF-IDF fallback")
            return tfidf_store

# Function to retrieve relevant context based on step instruction
def retrieve_context(vector_store, step_instruction, k=2):
    # Retrieve the most similar chunks to the step instruction
    relevant_chunks = vector_store.similarity_search(step_instruction, k=k)
    
    # Concatenate the content of the chunks
    context = "\n\n".join([doc.page_content for doc in relevant_chunks])
    
    return context

# Improved step instructions with more variety and natural language
STEP_INSTRUCTIONS_VARIANTS = {
    "Montage der Komponenten": [
        """
Führen Sie bitte folgende Montageschritte aus:

1. CARDIOHELP-i Halterung anbringen:
   - HKH 9102-M Halterung (für 38mm Mastdurchmesser) befestigen
   
2. Transport vorbereiten:
   - HKH 8860 Transporthalter bereitstellen
   - Emergency Drive sicher in der Transportbox verstauen
   
3. Notantrieb installieren:
   - HKH 9101-R/M Halterung für den Emergency Drive montieren
   
4. Hauptgerät installieren:
   - CARDIOHELP-i auf der Halterung sicher befestigen
   - Schutzbügel nach oben klappen
   - Stromkabel anschließen und Versorgung sicherstellen
        """,
        
        """
Montieren Sie das CARDIOHELP-System wie folgt:

1. Halterungsmontage:
   - Nehmen Sie die HKH 9102-M Halterung (38mm)
   - Befestigen Sie diese am vorgesehenen Mast
   
2. Transportsystem vorbereiten:
   - HKH 8860 Transporthalter positionieren
   - CARDIOHELP Emergency Drive in der Transportbox verstauen
   
3. Notantrieb-Vorbereitung:
   - Bringen Sie die HKH 9101-R/M Halterung an
   - Bereiten Sie den Notantrieb für die Montage vor
   
4. Hauptsystem aufbauen:
   - CARDIOHELP-i vorsichtig auf der Halterung platzieren
   - Entriegeln Sie den Schutzbügel und öffnen Sie ihn
   - Verbinden Sie das Gerät mit der Stromversorgung
        """
    ],
    
    "Anschluss von Sensoren und externen Systemen": [
        """
Schließen Sie Sensoren und externe Systeme wie folgt an:

1. Sensoren verbinden:
   - Achten Sie auf die roten Markierungen bei allen Steckern
   - Führen Sie die Stecker gerade ein, ohne zu verkanten
   
2. Externe Komponenten anschließen:
   - Datenaufzeichnungssystem mit dem USB-Typ-B Port verbinden
   - Externes Alarmsystem gemäß Kennzeichnung anschließen
   
3. Venösen Messkopf montieren:
   - Stecken Sie den Messkopf in den gekennzeichneten Anschluss
   - Setzen Sie ihn anschließend in die dafür vorgesehene Halterung
   
4. Einmalprodukt-Sensoren verbinden:
   - WICHTIG: Alle Sensorkabel VOR dem Einschalten anschließen
   - Auf korrekte Ausrichtung für die automatische Erkennung achten
        """,
        
        """
Verbinden Sie nun alle Sensoren und Systeme:

1. Sensoranschlüsse herstellen:
   - Orientieren Sie sich an den roten Markierungen auf allen Steckverbindungen
   - Stellen Sie sicher, dass alle Anschlüsse fest sitzen
   
2. Verbindung zu externen Geräten:
   - Schließen Sie das Datenaufzeichnungssystem über den USB-B-Port an
   - Verbinden Sie das externe Alarmierungssystem entsprechend den Markierungen
   
3. Venösen Messkopf einrichten:
   - Stecken Sie den Kopf in den "Venöser Messkopf"-Port
   - Rasten Sie ihn danach in der Halterung ein
   
4. Vorbereitung der Einmalprodukt-Sensorik:
   - Sensorkabel müssen unbedingt VOR dem Systemstart angeschlossen sein
   - Beachten Sie die korrekte Orientierung für optimale Funktionalität
        """
    ],
    
    "Einschalten und Systemkonfiguration": [
        """
Schalten Sie das System ein und konfigurieren Sie es:

1. Einschaltvorgang:
   - Überprüfen Sie, dass der venöse Messkopf korrekt positioniert ist
   - Drücken Sie die "Ein/Aus"-Taste und warten Sie auf den Selbsttest
   
2. Displayeinstellungen anpassen:
   - Stellen Sie die Bildschirmhelligkeit auf einen angenehmen Wert
   - Passen Sie die Systemlautstärke an die Umgebung an
   - Wählen Sie Ihre bevorzugte Sprache aus
   - Aktualisieren Sie Datum und Uhrzeit
   
3. Systemdiagnose:
   - Überprüfen Sie die installierte Software-Version
   - Kontrollieren Sie die angezeigten Betriebsstunden
   - Prüfen Sie den Ladezustand der Batterien
   
4. Erweiterte Einstellungen (mit Passwort):
   - Konfigurieren Sie die Alarmparameter nach Klinikstandard
   - Speichern Sie die personalisierten Einstellungen
        """,
        
        """
System starten und einrichten:

1. System aktivieren:
   - Stellen Sie sicher, dass der venöse Messkopf gesichert ist
   - Betätigen Sie den Einschaltknopf und beobachten Sie den Selbsttest
   
2. Basiskonfiguration durchführen:
   - Passen Sie die Displayhelligkeit an die Lichtverhältnisse an
   - Regulieren Sie die Alarmtonlautstärke
   - Stellen Sie die gewünschte Benutzersprache ein
   - Konfigurieren Sie die korrekte Zeit und das Datum
   
3. Systeminformationen überprüfen:
   - Notieren Sie die aktuelle Software-Version
   - Überprüfen Sie die aufgezeichneten Betriebsstunden
   - Kontrollieren Sie den Batteriestatus aller Komponenten
   
4. Administrator-Funktionen (Passworteingabe erforderlich):
   - Passen Sie die Alarmgrenzwerte an
   - Speichern Sie die klinikspezifischen Konfigurationen im System
        """
    ],
    
    "Anwendungsvorbereitung und Funktionstest": [
        """
Bereiten Sie die Anwendung vor und führen Sie Funktionstests durch:

1. Perfusionssystem vorbereiten:
   - Installieren Sie das Einmalprodukt entsprechend der Markierungen
   - Verbinden Sie alle integrierten Sensoren mit den Anschlüssen
   
2. Sensorkonfiguration:
   - Positionieren Sie den Fluss- und Blasensensor präzise
   - Bringen Sie die Temperatursensoren an den Messpunkten an
   - Installieren Sie die Drucksensoren an den vorgesehenen Stellen
   
3. Systemtests durchführen:
   - Testen Sie die Funktion der Blasenerkennung
   - Überprüfen Sie die Niveauüberwachung auf korrekte Reaktion
   - Kontrollieren Sie den Status aller Messköpfe
   
4. Sensorkalibrierung:
   - Führen Sie die Nullpunkt-Kalibrierung für alle Drucksensoren durch
   - Kalibrieren Sie den Fluss-Offset für präzise Messungen
   
5. Abschließende Sicherheitsprüfung:
   - Überprüfen Sie alle Sicherheitssiegel auf Unversehrtheit
   - Testen Sie die Funktion aller Bedienelemente
   - Kontrollieren Sie sämtliche Sensorfunktionen
   - Führen Sie einen Testlauf des Notantriebs durch
        """,
        
        """
Führen Sie die finalen Vorbereitungen und Tests durch:

1. Perfusionseinheit einrichten:
   - Montieren Sie das Einmalprodukt nach Herstellerangaben
   - Verbinden Sie die Sensorik mit den entsprechenden Ports
   
2. Sensorpositionierung:
   - Bringen Sie den kombinierten Fluss-/Blasensensor an
   - Platzieren Sie alle Temperatursensoren an den Messstellen
   - Installieren Sie die Drucksensoren gemäß Anleitung
   
3. Funktionsüberprüfung:
   - Testen Sie die Funktionalität der Blasendetektion
   - Prüfen Sie die Reaktion der Niveauüberwachung
   - Verifizieren Sie den Status des Messkopfsystems
   
4. Systemkalibrierung:
   - Führen Sie für alle Drucksensoren eine Nullkalibrierung durch
   - Justieren Sie den Fluss-Off-Set nach Herstellervorgaben
   
5. Finaler Sicherheitscheck:
   - Inspizieren Sie alle Sicherheitssiegel
   - Überprüfen Sie die Funktion sämtlicher Bedienelemente
   - Testen Sie alle verbundenen Sensoren
   - Verifizieren Sie die Funktionalität des Notantriebs
        """
    ]
}

# Initialize the model with appropriate settings
def initialize_llm():
    try:
        llm = Llama(
            model_path="em_german_7b_v01.Q8_0.gguf",  # Ensure this path is correct
            n_ctx=2048,
            verbose=False,
            n_batch=256,
            echo=False,
            temperature=0.7  # Increased temperature for more variety
        )
        print("LLM erfolgreich initialisiert")
        return llm
    except Exception as e:
        print(f"Fehler bei der LLM-Initialisierung: {e}")
        print("Verwende Fallback-Modus ohne LLM")
        return None

# Define the workflow steps directly from the manual sections
workflow_steps = [
    "Montage der Komponenten",
    "Anschluss von Sensoren und externen Systemen",
    "Einschalten und Systemkonfiguration",
    "Anwendungsvorbereitung und Funktionstest"
]

# Improved variety for user response interpretation
def interpret_user_response(user_input):
    """Function that checks for confirmation words in German with more sophistication"""
    user_input = user_input.lower().strip()
   
    # Comprehensive list of German confirmation words/phrases
    confirmation_words = [
        "fertig", "erledigt", "abgeschlossen", "ok", "ja", "jawohl",
        "gemacht", "geschafft", "bereit", "weiter", "klar", "verstanden",
        "in ordnung", "passt", "kann weitergehen", "nächster schritt",
        "alles klar", "geht weiter", "bin soweit", "bin bereit", 
        "nächster", "okay", "check", "gut", "geht", "läuft", "weiter geht's"
    ]
   
    # Check if any confirmation word is in the user input
    for word in confirmation_words:
        if word in user_input:
            return "yes"
   
    return "no"

# Collection of different motivational/transition messages for variety
TRANSITION_MESSAGES = [
    "Sehr gut! Gehen wir zum nächsten Schritt über.",
    "Ausgezeichnet! Der nächste Schritt wartet auf Sie.",
    "Gut gemacht! Fahren wir mit dem nächsten Schritt fort.",
    "Prima! Jetzt zum nächsten Schritt.",
    "Perfekt! Weiter geht's mit dem nächsten Punkt.",
    "Hervorragend! Nun zum nächsten Arbeitsschritt.",
    "Das haben Sie gut gemeistert! Kommen wir zum nächsten Schritt.",
    "Verstanden! Fahren wir mit dem nächsten Schritt fort.",
    "Alles klar! Gehen wir weiter zum nächsten Schritt."
]

# Collection of error messages for variety
ERROR_MESSAGES = [
    "Entschuldigung, ich habe Ihre Antwort nicht verstanden. Bitte bestätigen Sie, wenn Sie fertig sind.",
    "Ich konnte nicht erkennen, ob Sie den Schritt abgeschlossen haben. Bitte sagen Sie 'fertig', 'weiter' oder ähnliches.",
    "Ihre Antwort war unklar. Bitte teilen Sie mir mit, wenn Sie bereit für den nächsten Schritt sind.",
    "Könnten Sie bitte bestätigen, dass Sie den Schritt abgeschlossen haben?",
    "Ich bin mir nicht sicher, ob Sie fortfahren möchten. Bitte bestätigen Sie mit 'weiter', 'fertig' oder 'ok'."
]

# Collection of completion messages for variety
COMPLETION_MESSAGES = [
    "CARDIOHELP-Einrichtung erfolgreich abgeschlossen! Das Gerät ist nun einsatzbereit.",
    "Hervorragend! Die CARDIOHELP-Einrichtung ist vollständig abgeschlossen. Das System ist jetzt betriebsbereit.",
    "Alle Einrichtungsschritte wurden erfolgreich durchgeführt! Das CARDIOHELP-System kann nun verwendet werden.",
    "Glückwunsch! Die CARDIOHELP-Systemeinrichtung wurde erfolgreich abgeschlossen. Das Gerät ist einsatzbereit.",
    "Die Einrichtung des CARDIOHELP-Systems ist nun vollständig abgeschlossen. Das Gerät steht zur Verfügung."
]

# Main workflow with improved variety
def main():
    print("CARDIOHELP-Einrichtungsassistent\n")
    print("Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.")
    print("--------------------------------------------------\n")
    
    # Process manual content and create vector store
    chunks = process_manual_content()
    vector_store = create_vector_store(chunks)
    
    # Initialize LLM
    llm = initialize_llm()
    
    current_step = 0
    
    while current_step < len(workflow_steps):
        step_name = workflow_steps[current_step]
        
        # Get relevant content from the manual for this step
        context = retrieve_context(vector_store, step_name)
        
        # Default to a random variant from our predefined instructions
        instructions = random.choice(STEP_INSTRUCTIONS_VARIANTS[step_name])
        
        if llm:
            try:
                # Improved prompt engineering with less context that might confuse the LLM
                # and clearer instructions on what to generate
                system_prompt = f"""
                Du bist ein hilfreicher medizinischer Assistent, der bei der Einrichtung des CARDIOHELP-Systems unterstützt.
                
                Erstelle eine klare, präzise Anleitung für:
                Schritt {current_step + 1}: {step_name}
                
                Wichtig:
                1. Formuliere komplette, praktische Anweisungen in natürlichem Deutsch
                2. Verwende Keine nummerierte Listen für die Teilschritte, un nur eine verbinlichen Text.
                3. Beginne mit einer kurze Einleitungstext.
                4. Diese Anweisungen sollen immer anders formuliert werden aber präzizer seinn.
                5. Verwende keine Füllwörter oder allgemeine Floskeln.
                
                Relevante Information:
                {context}
                """
                
                # Generate a response
                response = llm(
                    system_prompt,
                    max_tokens=700,
                    temperature=0.7,  # Higher temperature for variety
                    stop=["</s>", "<s>", "Wichtig:", "Relevante Information:"],
                )
                
                llm_output = response['choices'][0]['text'].strip()
                
                # Improved validation to detect low-quality outputs
                low_quality_indicators = [
                    "Bitte gib", "Ich bin", "Ich werde", "Als Ihr", "Gerne erstelle",
                    "Hier ist", "Verstanden", "Ich kann", "Natürlich"
                ]
                
                # Check if the output is too short, contains meta-commentary, or is low quality
                if (len(llm_output) < 100 or
                    any(indicator in llm_output for indicator in low_quality_indicators) or
                    "Schritt" in llm_output[:20]):  # LLM repeating the step intro
                    
                    print("LLM-Antwort entspricht nicht den Qualitätsanforderungen - verwende vordefinierte Anweisung")
                else:
                    instructions = llm_output
                
            except Exception as e:
                print(f"Fehler bei der LLM-Antwort: {e}")
                # In case of error, fall back to our predefined variants
        
        # Display the instructions with better formatting
        print(f"\n--- Schritt {current_step + 1}: {step_name} ---")
        print(instructions.strip())
            
        # For demonstration - in a real app, use: user_input = input("> ")
        user_input = random.choice(["weiter", "fertig", "ok", "ja", "erledigt", "gemacht"])
        print(f"\n> {user_input}")
        
        # Check if user confirmed completion
        decision = interpret_user_response(user_input)
        if decision == "yes":
            current_step += 1
            if current_step < len(workflow_steps):
                # Use variable transition messages for more natural feeling
                print(f"\n{random.choice(TRANSITION_MESSAGES)}")
            else:
                # Use variable completion messages
                print(f"\n{random.choice(COMPLETION_MESSAGES)}")
        else:
            # Use variable error messages
            print(random.choice(ERROR_MESSAGES))
    
if __name__ == "__main__":
    main()

CARDIOHELP-Einrichtungsassistent

Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.
--------------------------------------------------

Bedienungsanleitung in 11 Teile aufgeteilt


C:\Users\NIKO9\AppData\Local\Temp\ipykernel_28588\4009766899.py:53: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\NIKO9\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vektordatenbank erfolgreich erstellt mit multilingual model


llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


LLM erfolgreich initialisiert
LLM-Antwort entspricht nicht den Qualitätsanforderungen - verwende vordefinierte Anweisung

--- Schritt 1: Montage der Komponenten ---
Führen Sie bitte folgende Montageschritte aus:

1. CARDIOHELP-i Halterung anbringen:
   - HKH 9102-M Halterung (für 38mm Mastdurchmesser) befestigen

2. Transport vorbereiten:
   - HKH 8860 Transporthalter bereitstellen
   - Emergency Drive sicher in der Transportbox verstauen

3. Notantrieb installieren:
   - HKH 9101-R/M Halterung für den Emergency Drive montieren

4. Hauptgerät installieren:
   - CARDIOHELP-i auf der Halterung sicher befestigen
   - Schutzbügel nach oben klappen
   - Stromkabel anschließen und Versorgung sicherstellen

> ok

Sehr gut! Gehen wir zum nächsten Schritt über.
LLM-Antwort entspricht nicht den Qualitätsanforderungen - verwende vordefinierte Anweisung

--- Schritt 2: Anschluss von Sensoren und externen Systemen ---
Verbinden Sie nun alle Sensoren und Systeme:

1. Sensoranschlüsse herstellen:
   

# Final RAG-PIPE

In [1]:
import random
from llama_cpp import Llama
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# The manual content (same as before)
MANUAL_CONTENT = """CARDIOHELP System-Vorbereitung in 4 Schritten
Schritt 1: Montage der Komponenten
Halterung für CARDIOHELP-i montieren: HKH 9102-M für 38mm Mastdurchmesser verwenden
Transportzubehör vorbereiten: Transporthalter HKH 8860, CARDIOHELP Emergency Drive in Transportbox sichern
CARDIOHELP Emergency Drive positionieren: Halterung für Notantrieb montieren (HKH 9101-R/M)
CARDIOHELP-i aufstellen: Gerät auf Halterung befestigen, Schutzbügel öffnen, Stromversorgung anschließen
Schritt 2: Anschluss von Sensoren und externen Systemen
Sensoren anschließen: Stecker anhand der roten Markierungen ausrichten
Externe Geräte verbinden: Datenaufzeichnungssystem über USB-Typ-B, externes Alarmsystem
Venösen Messkopf anschließen: An "Venöser Messkopf"-Anschluss und in Halterung platzieren
Sensorkabel für Einmalprodukt verbinden: Vor dem Einschalten anschließen für korrekte Erkennung
Schritt 3: Einschalten und Systemkonfiguration
Einschalten und Selbsttest: Venösen Messkopf vor Einschalten sichern, "Ein/Aus"-Taste drücken
Grundeinstellungen konfigurieren: Helligkeit, Lautstärke, Sprache, Zeit/Datum einstellen
Systeminformationen prüfen: Software-Version, Betriebsstunden, Batteriestatus kontrollieren
Key-User-Funktionen (passwortgeschützt): Alarmkonfiguration, Klinikkonfiguration speichern
Schritt 4: Anwendungsvorbereitung und Funktionstest
Perfusion vorbereiten: Einmalprodukt montieren, integrierte Sensorik anschließen
Erforderliche Sensoren positionieren: Fluss-/Blasensensor, Temperatursensoren, Drucksensoren
Funktionstests durchführen: Blasenüberwachung, Niveauüberwachung, Messkopf-Status prüfen
Kalibrierungen vornehmen: Nullkalibrierung für Drucksensoren, Fluss-Off-Set-Kalibrierung
Abschließende Überprüfung: Sicherheitssiegel, Bedienelemente, Sensorfunktionen, Notantrieb"""

# Structure the manual data for easier access
def structure_manual_data():
    step_data = {}
    current_step = None
    
    # Parse the manual content line by line
    for line in MANUAL_CONTENT.split('\n'):
        if line.startswith('Schritt '):
            # Extract step number and name
            parts = line.split(': ', 1)
            if len(parts) == 2:
                current_step = parts[1]
                step_data[current_step] = []
        elif current_step and line and not line.startswith('CARDIOHELP'):
            # Add step details
            step_data[current_step].append(line)
    
    return step_data

# Function to reformulate text with a variety of styles
def reformulate_instruction(step_name, substeps, style="direct"):
    """
    Reformulate instructions in different styles
    """
    # Prepare substeps from manual content
    formatted_substeps = []
    for i, substep in enumerate(substeps):
        parts = substep.split(': ', 1)
        if len(parts) == 2:
            action, details = parts[0], parts[1]
            formatted_substeps.append((action, details))
    
    # Different introduction templates
    intros = [
        f"Bei der {step_name} beachten Sie bitte folgende Schritte:",
        f"Für die {step_name} führen Sie diese Arbeitsschritte durch:",
        f"Gehen Sie für die {step_name} wie folgt vor:",
        f"Zur korrekten {step_name} befolgen Sie diese Anleitung:",
        f"Folgende Schritte sind für die {step_name} durchzuführen:",
        f"Die {step_name} erfolgt in diesen Schritten:",
        f"Für eine erfolgreiche {step_name} beachten Sie diese Punkte:"
    ]
    
    # Instruction verb variations
    action_verbs = [
        ("Montieren", "Befestigen", "Anbringen", "Installieren", "Positionieren"),
        ("Vorbereiten", "Bereitstellen", "Herrichten", "Organisieren", "Einrichten"),
        ("Positionieren", "Platzieren", "Ausrichten", "Einsetzen", "Aufstellen"),
        ("Aufstellen", "Platzieren", "Installieren", "Einrichten", "Montieren"),
        ("Anschließen", "Verbinden", "Einstecken", "Koppeln", "Anklemmen"),
        ("Verbinden", "Anschließen", "Koppeln", "Verknüpfen", "Zusammenführen"),
        ("Einschalten", "Aktivieren", "Starten", "In Betrieb nehmen", "Hochfahren"),
        ("Konfigurieren", "Einstellen", "Anpassen", "Justieren", "Einrichten"),
        ("Prüfen", "Kontrollieren", "Überprüfen", "Inspizieren", "Begutachten"),
        ("Vornehmen", "Durchführen", "Ausführen", "Erledigen", "Realisieren")
    ]
    
    # Format variations for improved variety
    bullet_styles = ["•", "➤", "✓", "-", "→"]
    
    # Choose a random introduction
    intro = random.choice(intros)
    
    # Generate the reformulated instructions
    reformulated = [intro, ""]
    
    for i, (action, details) in enumerate(formatted_substeps):
        # Find matching action verb and choose a synonym
        matching_verbs = None
        for verb_set in action_verbs:
            if any(verb.lower() in action.lower() for verb in verb_set):
                matching_verbs = verb_set
                break
        
        # If no matching verbs found, use the original action
        if matching_verbs:
            new_action = random.choice(matching_verbs)
        else:
            new_action = action
            
        # Choose a random bullet style
        bullet = random.choice(bullet_styles) if random.random() > 0.5 else f"{i+1}."
        
        # Reformat the details with some variations
        detail_parts = details.split(", ")
        if len(detail_parts) > 1 and random.random() > 0.5:
            # Sometimes format multi-part details as sub-bullets
            reformulated.append(f"{bullet} {new_action} Sie:")
            for part in detail_parts:
                reformulated.append(f"   - {part}")
        else:
            # Simple format
            reformulated.append(f"{bullet} {new_action} Sie {details}")
            
        # Add an empty line for readability if not the last item
        if i < len(formatted_substeps) - 1:
            reformulated.append("")
    
    return "\n".join(reformulated)

# Function to process the manual and create a vector store (similar to before)
def process_manual_content():
    doc = Document(page_content=MANUAL_CONTENT, metadata={"source": "CARDIOHELP-Manual"})
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", ":", " ", ""]
    )
    chunks = text_splitter.split_documents([doc])
    
    print(f"Bedienungsanleitung in {len(chunks)} Teile aufgeteilt")
    return chunks

def create_vector_store(chunks):
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
            model_kwargs={"device": "cpu"},
        )
        
        vector_store = FAISS.from_documents(chunks, embeddings)
        print("Vektordatenbank erfolgreich erstellt mit multilingual model")
        return vector_store
        
    except Exception as e:
        print(f"Fehler mit dem ersten Modell: {e}")
        
        try:
            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2",
                model_kwargs={"device": "cpu"},
            )
            
            vector_store = FAISS.from_documents(chunks, embeddings)
            print("Vektordatenbank erfolgreich erstellt mit alternativem Modell")
            return vector_store
            
        except Exception as e:
            print(f"Fehler mit dem zweiten Modell: {e}")
            
            from sklearn.feature_extraction.text import TfidfVectorizer
            from sklearn.metrics.pairwise import cosine_similarity
            
            texts = [doc.page_content for doc in chunks]
            
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(texts)
            
            class TfidfStore:
                def __init__(self, matrix, vectorizer, docs):
                    self.matrix = matrix
                    self.vectorizer = vectorizer
                    self.docs = docs
                
                def similarity_search(self, query, k=2):
                    query_vec = self.vectorizer.transform([query])
                    similarity = cosine_similarity(query_vec, self.matrix).flatten()
                    indices = similarity.argsort()[:-k-1:-1]
                    return [self.docs[i] for i in indices]
            
            tfidf_store = TfidfStore(tfidf_matrix, vectorizer, chunks)
            print("Vektordatenbank erfolgreich erstellt mit TF-IDF fallback")
            return tfidf_store

# LLM initialization function - kept for potential usage but will rely less on it
def initialize_llm():
    try:
        llm = Llama(
            model_path="em_german_7b_v01.Q8_0.gguf",
            n_ctx=2048,
            verbose=False,
            n_batch=256,
            echo=False,
            temperature=0.7
        )
        print("LLM erfolgreich initialisiert")
        return llm
    except Exception as e:
        print(f"Fehler bei der LLM-Initialisierung: {e}")
        print("Verwende Fallback-Modus ohne LLM")
        return None

# Define the workflow steps
workflow_steps = [
    "Montage der Komponenten",
    "Anschluss von Sensoren und externen Systemen",
    "Einschalten und Systemkonfiguration",
    "Anwendungsvorbereitung und Funktionstest"
]

# Interpret user response function (same as before)
def interpret_user_response(user_input):
    user_input = user_input.lower().strip()
   
    confirmation_words = [
        "fertig", "erledigt", "abgeschlossen", "ok", "ja", "jawohl",
        "gemacht", "geschafft", "bereit", "weiter", "klar", "verstanden",
        "in ordnung", "passt", "kann weitergehen", "nächster schritt",
        "alles klar", "geht weiter", "bin soweit", "bin bereit", 
        "nächster", "okay", "check", "gut", "geht", "läuft", "weiter geht's"
    ]
   
    for word in confirmation_words:
        if word in user_input:
            return "yes"
   
    return "no"

# Variety in user interaction responses
TRANSITION_MESSAGES = [
    "Sehr gut! Gehen wir zum nächsten Schritt über.",
    "Ausgezeichnet! Der nächste Schritt wartet auf Sie.",
    "Gut gemacht! Fahren wir mit dem nächsten Schritt fort.",
    "Prima! Jetzt zum nächsten Schritt.",
    "Perfekt! Weiter geht's mit dem nächsten Punkt.",
    "Hervorragend! Nun zum nächsten Arbeitsschritt.",
    "Das haben Sie gut gemeistert! Kommen wir zum nächsten Schritt.",
    "Verstanden! Fahren wir mit dem nächsten Schritt fort.",
    "Alles klar! Gehen wir weiter zum nächsten Schritt."
]

ERROR_MESSAGES = [
    "Entschuldigung, ich habe Ihre Antwort nicht verstanden. Bitte bestätigen Sie, wenn Sie fertig sind.",
    "Ich konnte nicht erkennen, ob Sie den Schritt abgeschlossen haben. Bitte sagen Sie 'fertig', 'weiter' oder ähnliches.",
    "Ihre Antwort war unklar. Bitte teilen Sie mir mit, wenn Sie bereit für den nächsten Schritt sind.",
    "Könnten Sie bitte bestätigen, dass Sie den Schritt abgeschlossen haben?",
    "Ich bin mir nicht sicher, ob Sie fortfahren möchten. Bitte bestätigen Sie mit 'weiter', 'fertig' oder 'ok'."
]

COMPLETION_MESSAGES = [
    "CARDIOHELP-Einrichtung erfolgreich abgeschlossen! Das Gerät ist nun einsatzbereit.",
    "Hervorragend! Die CARDIOHELP-Einrichtung ist vollständig abgeschlossen. Das System ist jetzt betriebsbereit.",
    "Alle Einrichtungsschritte wurden erfolgreich durchgeführt! Das CARDIOHELP-System kann nun verwendet werden.",
    "Glückwunsch! Die CARDIOHELP-Systemeinrichtung wurde erfolgreich abgeschlossen. Das Gerät ist einsatzbereit.",
    "Die Einrichtung des CARDIOHELP-Systems ist nun vollständig abgeschlossen. Das Gerät steht zur Verfügung."
]

# Main workflow function
def main():
    print("CARDIOHELP-Einrichtungsassistent\n")
    print("Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.")
    print("--------------------------------------------------\n")
    
    # Process manual data in structured format
    structured_data = structure_manual_data()
    
    # Also create the vector store (for compatibility with original code)
    chunks = process_manual_content()
    vector_store = create_vector_store(chunks)
    
    # Initialize LLM (but will rely less on it)
    llm = initialize_llm()
    
    current_step = 0
    
    while current_step < len(workflow_steps):
        step_name = workflow_steps[current_step]
        
        # Get the substeps for the current step
        substeps = structured_data.get(step_name, [])
        
        # Generate reformulated instructions
        instructions = reformulate_instruction(step_name, substeps)
        
        # Try using LLM only if we have full confidence
        if llm and random.random() < 0.2:  # Only try LLM 20% of the time to avoid wasting time on bad outputs
            try:
                system_prompt = f"""
                Als CARDIOHELP-Assistent geben Sie klare, strukturierte Anleitungen für:
                
                Schritt {current_step + 1}: {step_name}
                
                Wichtig:
                1. Formulieren Sie präzise Anweisungen in natürlichem Deutsch
                2. Verwenden Sie nummerierte Schritte
                3. Beginnen Sie mit einer kurzen Einleitung
                4. Vermeiden Sie Wiederholungen und Floskeln
                
                Details zum Schritt:
                {', '.join(substeps)}
                """
                
                response = llm(
                    system_prompt,
                    max_tokens=700,
                    temperature=0.3,
                    stop=["</s>", "<s>"],
                )
                
                llm_output = response['choices'][0]['text'].strip()
                
                # Strict quality check
                low_quality_indicators = [
                    "Bitte gib", "Ich bin", "Ich werde", "Als Ihr", "Gerne erstelle",
                    "Hier ist", "Verstanden", "Ich kann", "Natürlich", "würde ich"
                ]
                
                if (len(llm_output) > 200 and
                    not any(indicator in llm_output for indicator in low_quality_indicators)):
                    instructions = llm_output
                else:
                    print("LLM-Antwort wurde verworfen - verwende reformulierte Anweisung")
            except Exception as e:
                print(f"Fehler bei der LLM-Antwort: {e}")
        
        # Display the instructions
        print(f"\n ASISSTANT --- Schritt {current_step + 1}: {step_name} ---")
        print(instructions.strip())
            
        # For demonstration - in a real app, use: user_input = input("> ")
        user_input = random.choice(["weiter", "fertig", "ok", "ja", "erledigt", "gemacht"])
        print(f"\n USER> {user_input}")
        
        # Check if user confirmed completion
        decision = interpret_user_response(user_input)
        if decision == "yes":
            current_step += 1
            if current_step < len(workflow_steps):
                print(f"\n{random.choice(TRANSITION_MESSAGES)}")
            else:
                print(f"\n{random.choice(COMPLETION_MESSAGES)}")
        else:
            print(random.choice(ERROR_MESSAGES))
    
if __name__ == "__main__":
    main()

CARDIOHELP-Einrichtungsassistent

Folgen Sie den Anweisungen und sagen Sie 'weiter', 'fertig' oder 'ok', um zum nächsten Schritt zu gelangen.
--------------------------------------------------

Bedienungsanleitung in 11 Teile aufgeteilt


C:\Users\NIKO9\AppData\Local\Temp\ipykernel_28704\1546971082.py:149: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\NIKO9\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vektordatenbank erfolgreich erstellt mit multilingual model


llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


LLM erfolgreich initialisiert

 ASISSTANT --- Schritt 1: Montage der Komponenten ---
Für die Montage der Komponenten führen Sie diese Arbeitsschritte durch:

• Positionieren Sie HKH 9102-M für 38mm Mastdurchmesser verwenden

2. Einrichten Sie Transporthalter HKH 8860, CARDIOHELP Emergency Drive in Transportbox sichern

 USER> ok

Alles klar! Gehen wir weiter zum nächsten Schritt.

 ASISSTANT --- Schritt 2: Anschluss von Sensoren und externen Systemen ---
1. Anschließen der Sensoren:
                    - Stecker anhand der roten Markierungen ausrichten
                    - Externe Geräte verbinden:
                        - Datenaufzeichnungssystem über USB-Typ-B
                        - externes Alarmsystem
                    - Venösen Messkopf anschließen:
                        - An "Venöser Messkopf"-Anschluss und in Halterung platzieren
                    - Sensorkabel für Einmalprodukt verbinden:
                        - Vor dem Einschalten anschließen für korrekte Erkennun